# 前処理100本ノック（Polars）

In [1]:
# ライブラリのimport
import ibis
from ibis import _
import numpy as np
import polars as pl
from sklearn.model_selection import train_test_split, TimeSeriesSplit


ibis.options.interactive = True
ibis.set_backend("duckdb")

# DataFrameの作成
df_category = ibis.read_csv("../data/category.csv")
df_customer = ibis.read_csv("../data/customer.csv").cast({"gender_cd": str})
df_geocode = ibis.read_csv("../data/geocode.csv")
df_product = ibis.read_csv("../data/product.csv")
df_receipt = ibis.read_csv("../data/receipt.csv")
df_store = ibis.read_csv("../data/store.csv").cast({"prefecture_cd": str})

# 演習問題
## 表示

---
> P-001: レシート明細データ（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [2]:
df_receipt.head(10)

┏━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┓
┃ sales_ymd ┃ sales_epoch ┃ store_cd ┃ receipt_no ┃ receipt_sub_no ┃ customer_id    ┃ product_cd ┃ quantity ┃ amount ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━┩
│ int64     │ int64       │ string   │ int64      │ int64          │ string         │ string     │ int64    │ int64  │
├───────────┼─────────────┼──────────┼────────────┼────────────────┼────────────────┼────────────┼──────────┼────────┤
│  20181103 │  1541203200 │ S14006   │        112 │              1 │ CS006214000001 │ P070305012 │        1 │    158 │
│  20181118 │  1542499200 │ S13008   │       1132 │              2 │ CS008415000097 │ P070701017 │        1 │     81 │
│  20170712 │  1499817600 │ S14028   │       1102 │              1 │ CS028414000014 │ P060101005 │        1 │    170 │
│  20190205 │  1549324800 │ S14042   │       1132 │              1 │ ZZ000000000000 │ P050301001 │        1 │     25 │
│  20180821 │  1534809600 │ S14025   │       1102 │              2 │ CS025415000050 │ P060102007 │        1 │     90 │
│  20190605 │  1559692800 │ S13003   │       1112 │              1 │ CS003515000195 │ P050102002 │        1 │    138 │
│  20181205 │  1543968000 │ S14024   │       1102 │              2 │ CS024514000042 │ P080101005 │        1 │     30 │
│  20190922 │  1569110400 │ S14040   │       1102 │              1 │ CS040415000178 │ P070501004 │        1 │    128 │
│  20170504 │  1493856000 │ S13020   │       1112 │              2 │ ZZ000000000000 │ P071302010 │        1 │    770 │
│  20191010 │  1570665600 │ S14027   │       1102 │              1 │ CS027514000015 │ P071101003 │        1 │    680 │
└───────────┴─────────────┴──────────┴────────────┴────────────────┴────────────────┴────────────┴──────────┴────────┘

In [3]:
# スライスでも可能
df_receipt[:10]

┏━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┓
┃ sales_ymd ┃ sales_epoch ┃ store_cd ┃ receipt_no ┃ receipt_sub_no ┃ customer_id    ┃ product_cd ┃ quantity ┃ amount ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━┩
│ int64     │ int64       │ string   │ int64      │ int64          │ string         │ string     │ int64    │ int64  │
├───────────┼─────────────┼──────────┼────────────┼────────────────┼────────────────┼────────────┼──────────┼────────┤
│  20181103 │  1541203200 │ S14006   │        112 │              1 │ CS006214000001 │ P070305012 │        1 │    158 │
│  20181118 │  1542499200 │ S13008   │       1132 │              2 │ CS008415000097 │ P070701017 │        1 │     81 │
│  20170712 │  1499817600 │ S14028   │       1102 │              1 │ CS028414000014 │ P060101005 │        1 │    170 │
│  20190205 │  1549324800 │ S14042   │       1132 │              1 │ ZZ000000000000 │ P050301001 │        1 │     25 │
│  20180821 │  1534809600 │ S14025   │       1102 │              2 │ CS025415000050 │ P060102007 │        1 │     90 │
│  20190605 │  1559692800 │ S13003   │       1112 │              1 │ CS003515000195 │ P050102002 │        1 │    138 │
│  20181205 │  1543968000 │ S14024   │       1102 │              2 │ CS024514000042 │ P080101005 │        1 │     30 │
│  20190922 │  1569110400 │ S14040   │       1102 │              1 │ CS040415000178 │ P070501004 │        1 │    128 │
│  20170504 │  1493856000 │ S13020   │       1112 │              2 │ ZZ000000000000 │ P071302010 │        1 │    770 │
│  20191010 │  1570665600 │ S14027   │       1102 │              1 │ CS027514000015 │ P071101003 │        1 │    680 │
└───────────┴─────────────┴──────────┴────────────┴────────────────┴────────────────┴────────────┴──────────┴────────┘

---
> P-002: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。

In [4]:
# DataFrameから特定の列を選択する場合は.select()メソッドを用いる
# .select()メソッドの中身はリスト型で渡してもいい
df_receipt.select("sales_ymd", "customer_id", "product_cd", "amount").head(10)

┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃ sales_ymd ┃ customer_id    ┃ product_cd ┃ amount ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ int64     │ string         │ string     │ int64  │
├───────────┼────────────────┼────────────┼────────┤
│  20181103 │ CS006214000001 │ P070305012 │    158 │
│  20181118 │ CS008415000097 │ P070701017 │     81 │
│  20170712 │ CS028414000014 │ P060101005 │    170 │
│  20190205 │ ZZ000000000000 │ P050301001 │     25 │
│  20180821 │ CS025415000050 │ P060102007 │     90 │
│  20190605 │ CS003515000195 │ P050102002 │    138 │
│  20181205 │ CS024514000042 │ P080101005 │     30 │
│  20190922 │ CS040415000178 │ P070501004 │    128 │
│  20170504 │ ZZ000000000000 │ P071302010 │    770 │
│  20191010 │ CS027514000015 │ P071101003 │    680 │
└───────────┴────────────────┴────────────┴────────┘

---
> P-003: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、5件表示せよ。ただし、sales_ymdをsales_dateに項目名を変更しながら抽出すること。

In [5]:
# renameは辞書形式で行う（Polarsと逆で、{変更後: 変更前}）

df_003 = (
    df_receipt
    .select(["sales_ymd", "customer_id", "product_cd","amount"])
    .rename({"sales_date": "sales_ymd"})
)

df_003.head()

┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃ sales_date ┃ customer_id    ┃ product_cd ┃ amount ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ int64      │ string         │ string     │ int64  │
├────────────┼────────────────┼────────────┼────────┤
│   20181103 │ CS006214000001 │ P070305012 │    158 │
│   20181118 │ CS008415000097 │ P070701017 │     81 │
│   20170712 │ CS028414000014 │ P060101005 │    170 │
│   20190205 │ ZZ000000000000 │ P050301001 │     25 │
│   20180821 │ CS025415000050 │ P060102007 │     90 │
└────────────┴────────────────┴────────────┴────────┘

## 抽出

---
> P-004: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [6]:
df_004 = (
    df_receipt
    .select("sales_ymd", "customer_id", "product_cd", "amount")
    .filter(df_receipt["customer_id"] == "CS018205000001")
)

df_004.head()

┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃ sales_ymd ┃ customer_id    ┃ product_cd ┃ amount ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ int64     │ string         │ string     │ int64  │
├───────────┼────────────────┼────────────┼────────┤
│  20180911 │ CS018205000001 │ P071401012 │   2200 │
│  20180414 │ CS018205000001 │ P060104007 │    600 │
│  20170614 │ CS018205000001 │ P050206001 │    990 │
│  20170614 │ CS018205000001 │ P060702015 │    108 │
│  20190216 │ CS018205000001 │ P071005024 │    102 │
└───────────┴────────────────┴────────────┴────────┘

---
> P-005: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [7]:
# &や|で繋ぐときは()で囲ってあげること

df_005 = (
    df_receipt
    .select("sales_ymd", "customer_id", "product_cd", "amount")
    .filter(
        (df_receipt["customer_id"] == "CS018205000001")
        & (df_receipt["amount"] >= 1000)
    )
)

df_005.head()

┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃ sales_ymd ┃ customer_id    ┃ product_cd ┃ amount ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ int64     │ string         │ string     │ int64  │
├───────────┼────────────────┼────────────┼────────┤
│  20180911 │ CS018205000001 │ P071401012 │   2200 │
│  20190226 │ CS018205000001 │ P071401020 │   2200 │
│  20180911 │ CS018205000001 │ P071401005 │   1100 │
└───────────┴────────────────┴────────────┴────────┘

---
> P-006: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [8]:
df_006 = (
    df_receipt
    .select("sales_ymd", "customer_id", "product_cd", "quantity", "amount")
    .filter(
        (df_receipt["customer_id"] == "CS018205000001")
        & ((df_receipt["amount"] >= 1000) | (df_receipt["quantity"] > 5))
    )
)

df_006.head()

┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┓
┃ sales_ymd ┃ customer_id    ┃ product_cd ┃ quantity ┃ amount ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━┩
│ int64     │ string         │ string     │ int64    │ int64  │
├───────────┼────────────────┼────────────┼──────────┼────────┤
│  20180911 │ CS018205000001 │ P071401012 │        1 │   2200 │
│  20180414 │ CS018205000001 │ P060104007 │        6 │    600 │
│  20190226 │ CS018205000001 │ P071401020 │        1 │   2200 │
│  20180911 │ CS018205000001 │ P071401005 │        1 │   1100 │
└───────────┴────────────────┴────────────┴──────────┴────────┘

---
> P-007: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [9]:
df_007 = (
    df_receipt.select("sales_ymd", "customer_id", "product_cd", "amount")
    .filter(
        (df_receipt["customer_id"] == "CS018205000001")
        & ((df_receipt["amount"] >= 1000) & (df_receipt["amount"] <= 2000))
    )
)

df_007.head()

┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃ sales_ymd ┃ customer_id    ┃ product_cd ┃ amount ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ int64     │ string         │ string     │ int64  │
├───────────┼────────────────┼────────────┼────────┤
│  20180911 │ CS018205000001 │ P071401005 │   1100 │
└───────────┴────────────────┴────────────┴────────┘

In [10]:
df_007 = (
    df_receipt.select("sales_ymd", "customer_id", "product_cd", "amount")
    .filter(
        (df_receipt["customer_id"] == "CS018205000001")
        & (df_receipt["amount"].between(1000, 2000))
    )
)

df_007.head()

┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃ sales_ymd ┃ customer_id    ┃ product_cd ┃ amount ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ int64     │ string         │ string     │ int64  │
├───────────┼────────────────┼────────────┼────────┤
│  20180911 │ CS018205000001 │ P071401005 │   1100 │
└───────────┴────────────────┴────────────┴────────┘

---
> P-008: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [11]:
df_008 = (
    df_receipt.select("sales_ymd", "customer_id", "product_cd", "amount")
    .filter(
        (df_receipt["customer_id"] == "CS018205000001")
        & (df_receipt["product_cd"] != "P071401019")
    )
)

df_008

┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃ sales_ymd ┃ customer_id    ┃ product_cd ┃ amount ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ int64     │ string         │ string     │ int64  │
├───────────┼────────────────┼────────────┼────────┤
│  20180911 │ CS018205000001 │ P071401012 │   2200 │
│  20180414 │ CS018205000001 │ P060104007 │    600 │
│  20170614 │ CS018205000001 │ P050206001 │    990 │
│  20170614 │ CS018205000001 │ P060702015 │    108 │
│  20190216 │ CS018205000001 │ P071005024 │    102 │
│  20180414 │ CS018205000001 │ P071101002 │    278 │
│  20190226 │ CS018205000001 │ P070902035 │    168 │
│  20190924 │ CS018205000001 │ P060805001 │    495 │
│  20190226 │ CS018205000001 │ P071401020 │   2200 │
│  20180911 │ CS018205000001 │ P071401005 │   1100 │
│         … │ …              │ …          │      … │
└───────────┴────────────────┴────────────┴────────┘

---
> P-009: 以下のPandas処理において、出力結果を変えずにORをANDに書き換え、Ibisで表現せよ。
> 
> pandas:`df_store.query('not(prefecture_cd == "13" | floor_area > 900)')` <br>

In [12]:
df_009 = df_store.filter(
    (df_store["prefecture_cd"] != "13")
    & (df_store["floor_area"] <= 900)
)

df_009

┏━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ store_cd ┃ store_name ┃ prefecture_cd ┃ prefecture ┃ address           ┃ address_kana                   ┃ tel_no       ┃ longitude ┃ latitude ┃ floor_area ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━┩
│ string   │ string     │ string        │ string     │ string            │ string                         │ string       │ float64   │ float64  │ float64    │
├──────────┼────────────┼───────────────┼────────────┼───────────────────┼────────────────────────────────┼──────────────┼───────────┼──────────┼────────────┤
│ S14046   │ 北山田店   │ 14            │ 神奈川県   │ 神奈川県横浜市都… │ カナガワケンヨコハマシツヅキ … │ 045-123-4049 │  139.5916 │ 35.56189 │      831.0 │
│ S14011   │ 日吉本町店 │ 14            │ 神奈川県   │ 神奈川県横浜市港… │ カナガワケンヨコハマシコウホ … │ 045-123-4033 │  139.6316 │ 35.54655 │      890.0 │
│ S12013   │ 習志野店   │ 12            │ 千葉県     │ 千葉県習志野市芝… │ チバケンナラシノシシバゾノイ … │ 047-123-4002 │  140.0220 │ 35.66122 │      808.0 │
└──────────┴────────────┴───────────────┴────────────┴───────────────────┴────────────────────────────────┴──────────────┴───────────┴──────────┴────────────┘

---
> P-010: 店舗データ（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、5件表示せよ。

In [13]:
df_010 = df_store.filter(df_store["store_cd"].startswith("S14"))

df_010.head()

┏━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ store_cd ┃ store_name ┃ prefecture_cd ┃ prefecture ┃ address             ┃ address_kana                  ┃ tel_no       ┃ longitude ┃ latitude ┃ floor_area ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━┩
│ string   │ string     │ string        │ string     │ string              │ string                        │ string       │ float64   │ float64  │ float64    │
├──────────┼────────────┼───────────────┼────────────┼─────────────────────┼───────────────────────────────┼──────────────┼───────────┼──────────┼────────────┤
│ S14010   │ 菊名店     │ 14            │ 神奈川県   │ 神奈川県横浜市港北… │ カナガワケンヨコハマシコウホ… │ 045-123-4032 │  139.6326 │ 35.50049 │     1732.0 │
│ S14033   │ 阿久和店   │ 14            │ 神奈川県   │ 神奈川県横浜市瀬谷… │ カナガワケンヨコハマシセヤク… │ 045-123-4043 │  139.4961 │ 35.45918 │     1495.0 │
│ S14036   │ 相模原中 … │ 14            │ 神奈川県   │ 神奈川県相模原市中… │ カナガワケンサガミハラシチュ… │ 042-123-4045 │  139.3716 │ 35.57327 │     1679.0 │
│ S14040   │ 長津田店   │ 14            │ 神奈川県   │ 神奈川県横浜市緑区… │ カナガワケンヨコハマシミドリ… │ 045-123-4046 │  139.4994 │ 35.52398 │     1548.0 │
│ S14050   │ 阿久和西店 │ 14            │ 神奈川県   │ 神奈川県横浜市瀬谷… │ カナガワケンヨコハマシセヤク… │ 045-123-4053 │  139.4961 │ 35.45918 │     1830.0 │
└──────────┴────────────┴───────────────┴────────────┴─────────────────────┴───────────────────────────────┴──────────────┴───────────┴──────────┴────────────┘

---
> P-011: 顧客データ（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、5件表示せよ。

In [14]:
df_011 = df_customer.filter(df_customer["customer_id"].endswith(1))

df_011.head()

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ customer_id    ┃ customer_name ┃ gender_cd ┃ gender ┃ birth_day  ┃ age   ┃ postal_cd ┃ address                 ┃ application_store_cd ┃ application_date ┃ status_cd    ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ string         │ string        │ string    │ string │ date       │ int64 │ string    │ string                  │ string               │ int64            │ string       │
├────────────────┼───────────────┼───────────┼────────┼────────────┼───────┼───────────┼─────────────────────────┼──────────────────────┼──────────────────┼──────────────┤
│ CS037613000071 │ 六角 雅彦     │ 9         │ 不明   │ 1952-04-01 │    66 │ 136-0076  │ 東京都江東区南砂******… │ S13037               │         20150414 │ 0-00000000-0 │
│ CS028811000001 │ 堀井 かおり   │ 1         │ 女性   │ 1933-03-27 │    86 │ 245-0016  │ 神奈川県横浜市泉区和泉… │ S14028               │         20160115 │ 0-00000000-0 │
│ CS040412000191 │ 川井 郁恵     │ 1         │ 女性   │ 1977-01-05 │    42 │ 226-0021  │ 神奈川県横浜市緑区北八… │ S14040               │         20151101 │ 1-20091025-4 │
│ CS028314000011 │ 小菅 あおい   │ 1         │ 女性   │ 1983-11-26 │    35 │ 246-0038  │ 神奈川県横浜市瀬谷区宮… │ S14028               │         20151123 │ 1-20080426-5 │
│ CS039212000051 │ 藤島 恵梨香   │ 1         │ 女性   │ 1997-02-03 │    22 │ 166-0001  │ 東京都杉並区阿佐谷北**… │ S13039               │         20171121 │ 1-20100215-4 │
└────────────────┴───────────────┴───────────┴────────┴────────────┴───────┴───────────┴─────────────────────────┴──────────────────────┴──────────────────┴──────────────┘

---
> P-012: 店舗データ（df_store）から、住所 (address) に"横浜市"が含まれるものだけ全項目表示せよ。

In [15]:
df_012 = df_store.filter(df_store["address"].contains("横浜市"))

df_012

┏━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ store_cd ┃ store_name ┃ prefecture_cd ┃ prefecture ┃ address             ┃ address_kana                   ┃ tel_no       ┃ longitude ┃ latitude ┃ floor_area ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━┩
│ string   │ string     │ string        │ string     │ string              │ string                         │ string       │ float64   │ float64  │ float64    │
├──────────┼────────────┼───────────────┼────────────┼─────────────────────┼────────────────────────────────┼──────────────┼───────────┼──────────┼────────────┤
│ S14010   │ 菊名店     │ 14            │ 神奈川県   │ 神奈川県横浜市港北… │ カナガワケンヨコハマシコウホ … │ 045-123-4032 │  139.6326 │ 35.50049 │     1732.0 │
│ S14033   │ 阿久和店   │ 14            │ 神奈川県   │ 神奈川県横浜市瀬谷… │ カナガワケンヨコハマシセヤク … │ 045-123-4043 │  139.4961 │ 35.45918 │     1495.0 │
│ S14040   │ 長津田店   │ 14            │ 神奈川県   │ 神奈川県横浜市緑区… │ カナガワケンヨコハマシミドリ … │ 045-123-4046 │  139.4994 │ 35.52398 │     1548.0 │
│ S14050   │ 阿久和西店 │ 14            │ 神奈川県   │ 神奈川県横浜市瀬谷… │ カナガワケンヨコハマシセヤク … │ 045-123-4053 │  139.4961 │ 35.45918 │     1830.0 │
│ S14028   │ 二ツ橋店   │ 14            │ 神奈川県   │ 神奈川県横浜市瀬谷… │ カナガワケンヨコハマシセヤク … │ 045-123-4042 │  139.4963 │ 35.46304 │     1574.0 │
│ S14012   │ 本牧和田店 │ 14            │ 神奈川県   │ 神奈川県横浜市中区… │ カナガワケンヨコハマシナカク … │ 045-123-4034 │  139.6582 │ 35.42156 │     1341.0 │
│ S14046   │ 北山田店   │ 14            │ 神奈川県   │ 神奈川県横浜市都筑… │ カナガワケンヨコハマシツヅキ … │ 045-123-4049 │  139.5916 │ 35.56189 │      831.0 │
│ S14011   │ 日吉本町店 │ 14            │ 神奈川県   │ 神奈川県横浜市港北… │ カナガワケンヨコハマシコウホ … │ 045-123-4033 │  139.6316 │ 35.54655 │      890.0 │
│ S14048   │ 中川中央店 │ 14            │ 神奈川県   │ 神奈川県横浜市都筑… │ カナガワケンヨコハマシツヅキ … │ 045-123-4051 │  139.5758 │ 35.54912 │     1657.0 │
│ S14042   │ 新山下店   │ 14            │ 神奈川県   │ 神奈川県横浜市中区… │ カナガワケンヨコハマシナカク … │ 045-123-4047 │  139.6593 │ 35.43894 │     1044.0 │
│ …        │ …          │ …             │ …          │ …                   │ …                              │ …            │         … │        … │          … │
└──────────┴────────────┴───────────────┴────────────┴─────────────────────┴────────────────────────────────┴──────────────┴───────────┴──────────┴────────────┘

---
> P-013: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、5件表示せよ。

先頭の文字は`^`で表す(https://qiita.com/luohao0404/items/7135b2b96f9b0b196bf3 参照)

In [16]:
df_013 = df_customer.filter(df_customer["status_cd"].re_search(r"^[A-F]"))

df_013.head()

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ customer_id    ┃ customer_name ┃ gender_cd ┃ gender ┃ birth_day  ┃ age   ┃ postal_cd ┃ address                  ┃ application_store_cd ┃ application_date ┃ status_cd    ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ string         │ string        │ string    │ string │ date       │ int64 │ string    │ string                   │ string               │ int64            │ string       │
├────────────────┼───────────────┼───────────┼────────┼────────────┼───────┼───────────┼──────────────────────────┼──────────────────────┼──────────────────┼──────────────┤
│ CS031415000172 │ 宇多田 貴美子 │ 1         │ 女性   │ 1976-10-04 │    42 │ 151-0053  │ 東京都渋谷区代々木*****… │ S13031               │         20150529 │ D-20100325-C │
│ CS015414000103 │ 奥野 陽子     │ 1         │ 女性   │ 1977-08-09 │    41 │ 136-0073  │ 東京都江東区北砂*******… │ S13015               │         20150722 │ B-20100609-B │
│ CS011215000048 │ 芦田 沙耶     │ 1         │ 女性   │ 1992-02-01 │    27 │ 223-0062  │ 神奈川県横浜市港北区日 … │ S14011               │         20150228 │ C-20100421-9 │
│ CS029415000023 │ 梅田 里穂     │ 1         │ 女性   │ 1976-01-17 │    43 │ 279-0043  │ 千葉県浦安市富士見*****… │ S12029               │         20150610 │ D-20100918-E │
│ CS035415000029 │ 寺沢 真希     │ 9         │ 不明   │ 1977-09-27 │    41 │ 158-0096  │ 東京都世田谷区玉川台***… │ S13035               │         20141220 │ F-20101029-F │
└────────────────┴───────────────┴───────────┴────────┴────────────┴───────┴───────────┴──────────────────────────┴──────────────────────┴──────────────────┴──────────────┘

---
> P-014: 顧客データ（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、5件表示せよ。

最後尾の文字は`$`で表す

In [17]:
df_014 = df_customer.filter(df_customer["status_cd"].re_search(r"[1-9]$"))

df_014.head()

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ customer_id    ┃ customer_name ┃ gender_cd ┃ gender ┃ birth_day  ┃ age   ┃ postal_cd ┃ address                  ┃ application_store_cd ┃ application_date ┃ status_cd    ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ string         │ string        │ string    │ string │ date       │ int64 │ string    │ string                   │ string               │ int64            │ string       │
├────────────────┼───────────────┼───────────┼────────┼────────────┼───────┼───────────┼──────────────────────────┼──────────────────────┼──────────────────┼──────────────┤
│ CS001215000145 │ 田崎 美紀     │ 1         │ 女性   │ 1995-03-29 │    24 │ 144-0055  │ 東京都大田区仲六郷*****… │ S13001               │         20170605 │ 6-20090929-2 │
│ CS033513000180 │ 安斎 遥       │ 1         │ 女性   │ 1962-07-11 │    56 │ 241-0823  │ 神奈川県横浜市旭区善部 … │ S14033               │         20150728 │ 6-20080506-5 │
│ CS011215000048 │ 芦田 沙耶     │ 1         │ 女性   │ 1992-02-01 │    27 │ 223-0062  │ 神奈川県横浜市港北区日 … │ S14011               │         20150228 │ C-20100421-9 │
│ CS040412000191 │ 川井 郁恵     │ 1         │ 女性   │ 1977-01-05 │    42 │ 226-0021  │ 神奈川県横浜市緑区北八 … │ S14040               │         20151101 │ 1-20091025-4 │
│ CS009315000023 │ 皆川 文世     │ 1         │ 女性   │ 1980-04-15 │    38 │ 154-0012  │ 東京都世田谷区駒沢*****… │ S13009               │         20150319 │ 5-20080322-1 │
└────────────────┴───────────────┴───────────┴────────┴────────────┴───────┴───────────┴──────────────────────────┴──────────────────────┴──────────────────┴──────────────┘

---
> P-015: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、5件表示せよ。

In [18]:
df_015 = df_customer.filter(df_customer["status_cd"].re_search(r"^[A-F].*[1-9]$"))

df_015.head()

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ customer_id    ┃ customer_name ┃ gender_cd ┃ gender ┃ birth_day  ┃ age   ┃ postal_cd ┃ address                  ┃ application_store_cd ┃ application_date ┃ status_cd    ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ string         │ string        │ string    │ string │ date       │ int64 │ string    │ string                   │ string               │ int64            │ string       │
├────────────────┼───────────────┼───────────┼────────┼────────────┼───────┼───────────┼──────────────────────────┼──────────────────────┼──────────────────┼──────────────┤
│ CS011215000048 │ 芦田 沙耶     │ 1         │ 女性   │ 1992-02-01 │    27 │ 223-0062  │ 神奈川県横浜市港北区日 … │ S14011               │         20150228 │ C-20100421-9 │
│ CS022513000105 │ 島村 貴美子   │ 1         │ 女性   │ 1962-03-12 │    57 │ 249-0002  │ 神奈川県逗子市山の根***… │ S14022               │         20150320 │ A-20091115-7 │
│ CS001515000096 │ 水野 陽子     │ 9         │ 不明   │ 1960-11-29 │    58 │ 144-0053  │ 東京都大田区蒲田本町***… │ S13001               │         20150614 │ A-20100724-7 │
│ CS013615000053 │ 西脇 季衣     │ 1         │ 女性   │ 1953-10-18 │    65 │ 261-0026  │ 千葉県千葉市美浜区幕張 … │ S12013               │         20150128 │ B-20100329-6 │
│ CS020412000161 │ 小宮 薫       │ 1         │ 女性   │ 1974-05-21 │    44 │ 174-0042  │ 東京都板橋区東坂下*****… │ S13020               │         20150822 │ B-20081021-3 │
└────────────────┴───────────────┴───────────┴────────┴────────────┴───────┴───────────┴──────────────────────────┴──────────────────────┴──────────────────┴──────────────┘

---
> P-016: 店舗データ（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [19]:
df_016 = df_store.filter(df_store["tel_no"].re_search("[0-9]{3}-[0-9]{3}-[0-9]{4}"))

df_016

┏━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ store_cd ┃ store_name ┃ prefecture_cd ┃ prefecture ┃ address             ┃ address_kana                  ┃ tel_no       ┃ longitude ┃ latitude ┃ floor_area ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━┩
│ string   │ string     │ string        │ string     │ string              │ string                        │ string       │ float64   │ float64  │ float64    │
├──────────┼────────────┼───────────────┼────────────┼─────────────────────┼───────────────────────────────┼──────────────┼───────────┼──────────┼────────────┤
│ S12014   │ 千草台店   │ 12            │ 千葉県     │ 千葉県千葉市稲毛区… │ チバケンチバシイナゲクチグサ… │ 043-123-4003 │  140.1180 │ 35.63559 │     1698.0 │
│ S13002   │ 国分寺店   │ 13            │ 東京都     │ 東京都国分寺市本多… │ トウキョウトコクブンジシホン… │ 042-123-4008 │  139.4802 │ 35.70566 │     1735.0 │
│ S14010   │ 菊名店     │ 14            │ 神奈川県   │ 神奈川県横浜市港北… │ カナガワケンヨコハマシコウホ… │ 045-123-4032 │  139.6326 │ 35.50049 │     1732.0 │
│ S14033   │ 阿久和店   │ 14            │ 神奈川県   │ 神奈川県横浜市瀬谷… │ カナガワケンヨコハマシセヤク… │ 045-123-4043 │  139.4961 │ 35.45918 │     1495.0 │
│ S14036   │ 相模原中 … │ 14            │ 神奈川県   │ 神奈川県相模原市中… │ カナガワケンサガミハラシチュ… │ 042-123-4045 │  139.3716 │ 35.57327 │     1679.0 │
│ S14040   │ 長津田店   │ 14            │ 神奈川県   │ 神奈川県横浜市緑区… │ カナガワケンヨコハマシミドリ… │ 045-123-4046 │  139.4994 │ 35.52398 │     1548.0 │
│ S14050   │ 阿久和西店 │ 14            │ 神奈川県   │ 神奈川県横浜市瀬谷… │ カナガワケンヨコハマシセヤク… │ 045-123-4053 │  139.4961 │ 35.45918 │     1830.0 │
│ S13052   │ 森野店     │ 13            │ 東京都     │ 東京都町田市森野三… │ トウキョウトマチダシモリノサ… │ 042-123-4030 │  139.4383 │ 35.55293 │     1087.0 │
│ S14028   │ 二ツ橋店   │ 14            │ 神奈川県   │ 神奈川県横浜市瀬谷… │ カナガワケンヨコハマシセヤク… │ 045-123-4042 │  139.4963 │ 35.46304 │     1574.0 │
│ S14012   │ 本牧和田店 │ 14            │ 神奈川県   │ 神奈川県横浜市中区… │ カナガワケンヨコハマシナカク… │ 045-123-4034 │  139.6582 │ 35.42156 │     1341.0 │
│ …        │ …          │ …             │ …          │ …                   │ …                             │ …            │         … │        … │          … │
└──────────┴────────────┴───────────────┴────────────┴─────────────────────┴───────────────────────────────┴──────────────┴───────────┴──────────┴────────────┘

## ソート

---
> P-017: 顧客データ（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭から全項目を5件表示せよ。


In [20]:
df_017 = df_customer.order_by("birth_day")

df_017.head()

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ customer_id    ┃ customer_name ┃ gender_cd ┃ gender ┃ birth_day  ┃ age   ┃ postal_cd ┃ address                ┃ application_store_cd ┃ application_date ┃ status_cd    ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ string         │ string        │ string    │ string │ date       │ int64 │ string    │ string                 │ string               │ int64            │ string       │
├────────────────┼───────────────┼───────────┼────────┼────────────┼───────┼───────────┼────────────────────────┼──────────────────────┼──────────────────┼──────────────┤
│ CS003813000014 │ 村山 菜々美   │ 1         │ 女性   │ 1928-11-26 │    90 │ 182-0007  │ 東京都調布市菊野台***… │ S13003               │         20160214 │ 0-00000000-0 │
│ CS026813000004 │ 吉村 朝陽     │ 1         │ 女性   │ 1928-12-14 │    90 │ 251-0043  │ 神奈川県藤沢市辻堂元 … │ S14026               │         20150723 │ 0-00000000-0 │
│ CS018811000003 │ 熊沢 美里     │ 1         │ 女性   │ 1929-01-07 │    90 │ 204-0004  │ 東京都清瀬市野塩*****… │ S13018               │         20150403 │ 0-00000000-0 │
│ CS027803000004 │ 内村 拓郎     │ 0         │ 男性   │ 1929-01-12 │    90 │ 251-0031  │ 神奈川県藤沢市鵠沼藤 … │ S14027               │         20151227 │ 0-00000000-0 │
│ CS013801000003 │ 天野 拓郎     │ 0         │ 男性   │ 1929-01-15 │    90 │ 274-0824  │ 千葉県船橋市前原東***… │ S12013               │         20160120 │ 0-00000000-0 │
└────────────────┴───────────────┴───────────┴────────┴────────────┴───────┴───────────┴────────────────────────┴──────────────────────┴──────────────────┴──────────────┘

---
> P-018: 顧客データ（df_customer）を生年月日（birth_day）で若い順にソートし、先頭から全項目を5件表示せよ。

In [21]:
# ibis.desc()で降順

df_018 = df_customer.order_by(ibis.desc("birth_day"))

df_018.head()

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ customer_id    ┃ customer_name ┃ gender_cd ┃ gender ┃ birth_day  ┃ age   ┃ postal_cd ┃ address               ┃ application_store_cd ┃ application_date ┃ status_cd    ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ string         │ string        │ string    │ string │ date       │ int64 │ string    │ string                │ string               │ int64            │ string       │
├────────────────┼───────────────┼───────────┼────────┼────────────┼───────┼───────────┼───────────────────────┼──────────────────────┼──────────────────┼──────────────┤
│ CS035114000004 │ 大村 美里     │ 1         │ 女性   │ 2007-11-25 │    11 │ 156-0053  │ 東京都世田谷区桜****… │ S13035               │         20150619 │ 6-20091205-6 │
│ CS022103000002 │ 福山 はじめ   │ 9         │ 不明   │ 2007-10-02 │    11 │ 249-0006  │ 神奈川県逗子市逗子**… │ S14022               │         20160909 │ 0-00000000-0 │
│ CS002113000009 │ 柴田 真悠子   │ 1         │ 女性   │ 2007-09-17 │    11 │ 184-0014  │ 東京都小金井市貫井南… │ S13002               │         20160304 │ 0-00000000-0 │
│ CS004115000014 │ 松井 京子     │ 1         │ 女性   │ 2007-08-09 │    11 │ 165-0031  │ 東京都中野区上鷺宮**… │ S13004               │         20161120 │ 1-20081231-1 │
│ CS002114000010 │ 山内 遥       │ 1         │ 女性   │ 2007-06-03 │    11 │ 184-0015  │ 東京都小金井市貫井北… │ S13002               │         20160920 │ 6-20100510-1 │
└────────────────┴───────────────┴───────────┴────────┴────────────┴───────┴───────────┴───────────────────────┴──────────────────────┴──────────────────┴──────────────┘

---
> P-019: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

- `average`:同じ値のものはランクの平均値になる(例:1, 2, 3, 3, 4, 4, 5) -> rank(1, 2, 3.5, 3.5, 5.5, 5.5, 7)
- `min`:同じ値のものはランクは最小値で表す(例:1, 2, 3, 3, 4, 4, 5) -> rank(1, 2, 3, 3, 5, 5, 7)
- `max`:同じ値のものはランクは最大値で表す(例:1, 2, 3, 3, 4, 4, 5) -> rank(1, 2, 4, 4, 6, 6, 7)
- `dense`:`min`と似ているが、割り当てられるランクはその前のランクの次の値になる(例:1, 2, 3, 3, 4, 4, 5) -> rank(1, 2, 3, 3, 4, 4, 5)
- `ordinal`:`min`と似ているが、同値は値の出現順で重複なしでランクを与えられる
- `random`:`ordinal`と似ているが、同値に与えられるランクはランダム

In [22]:
df_019 = (
    df_receipt
    .select("customer_id", "amount")
    # .mutate(rank=ibis.dense_rank().over(ibis.window(order_by=df_receipt["amount"])))
    .to_polars()
    .with_columns(
        pl.col("amount")
        .rank(method="min", descending=True)
        .alias("ranking")
    )
    .sort("ranking")
)

df_019

customer_id,amount,ranking
str,i64,u32
"""CS011415000006""",10925,1
"""ZZ000000000000""",6800,2
"""CS028605000002""",5780,3
"""CS015515000034""",5480,4
"""ZZ000000000000""",5480,4
…,…,…
"""CS011115000005""",10,104512
"""CS022514000024""",10,104512
"""ZZ000000000000""",10,104512


---
> P-020: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

## 集計

---
> P-021: レシート明細データ（df_receipt）に対し、件数をカウントせよ。

In [23]:
df_receipt.count().execute()  # .to_pandas()つけると値が取得できる

104681

---
> P-022: レシート明細データ（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [24]:
df_receipt.select("customer_id").nunique().execute()

8307

---
> P-023: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [25]:
df_023 = (
    df_receipt
    .group_by("store_cd")
    # _を利用して表示することも可能
    # .agg(sum_amount=_["amount"].sum(), sum_quantity=_["quantity"].sum())
    .aggregate(
        sum_amount=df_receipt["amount"].sum(),
        sum_quantity=df_receipt["quantity"].sum()
    )
    .order_by("store_cd")
)

df_023

┏━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ store_cd ┃ sum_amount ┃ sum_quantity ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ string   │ int64      │ int64        │
├──────────┼────────────┼──────────────┤
│ S12007   │     638761 │         2099 │
│ S12013   │     787513 │         2425 │
│ S12014   │     725167 │         2358 │
│ S12029   │     794741 │         2555 │
│ S12030   │     684402 │         2403 │
│ S13001   │     811936 │         2347 │
│ S13002   │     727821 │         2340 │
│ S13003   │     764294 │         2197 │
│ S13004   │     779373 │         2390 │
│ S13005   │     629876 │         2004 │
│ …        │          … │            … │
└──────────┴────────────┴──────────────┘

---
> P-024: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）を求め、5件表示せよ。

In [26]:
df_024 = (
    df_receipt
    .group_by("customer_id")
    .aggregate(newest_sales_ymd=df_receipt["sales_ymd"].max())
)

df_024.head(5)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ customer_id    ┃ newest_sales_ymd ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ string         │ int64            │
├────────────────┼──────────────────┤
│ ZZ000000000000 │         20191031 │
│ CS003515000195 │         20190605 │
│ CS024514000042 │         20181205 │
│ CS026515000042 │         20190904 │
│ CS005513000061 │         20170826 │
└────────────────┴──────────────────┘

---
> P-025: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上年月日（sales_ymd）を求め、10件表示せよ。

In [27]:
df_025 = (
    df_receipt.group_by("customer_id")
    .aggregate(oldest_sales_ymd=df_receipt["sales_ymd"].min())
)

df_025.head(5)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ customer_id    ┃ oldest_sales_ymd ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ string         │ int64            │
├────────────────┼──────────────────┤
│ ZZ000000000000 │         20170101 │
│ CS003515000195 │         20170505 │
│ CS024514000042 │         20170624 │
│ CS026515000042 │         20171117 │
│ CS005513000061 │         20170107 │
└────────────────┴──────────────────┘

---
> P-026: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）と古い売上年月日を求め、両者が異なるデータを10件表示せよ。

In [28]:
df_026 = (
    df_receipt
    .group_by("customer_id")
    .aggregate(
        sales_ymd_min=df_receipt["sales_ymd"].min(),
        sales_ymd_max=df_receipt["sales_ymd"].max()
    )
    .filter(_["sales_ymd_min"] != _["sales_ymd_max"])
)

df_026.head(10)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ customer_id    ┃ sales_ymd_min ┃ sales_ymd_max ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ string         │ int64         │ int64         │
├────────────────┼───────────────┼───────────────┤
│ ZZ000000000000 │      20170101 │      20191031 │
│ CS003515000195 │      20170505 │      20190605 │
│ CS024514000042 │      20170624 │      20181205 │
│ CS026515000042 │      20171117 │      20190904 │
│ CS005513000061 │      20170107 │      20170826 │
│ CS022414000065 │      20170227 │      20190625 │
│ CS030415000197 │      20170430 │      20190925 │
│ CS004515000723 │      20180906 │      20190822 │
│ CS008414000013 │      20170217 │      20190825 │
│ CS032215000073 │      20170913 │      20190224 │
└────────────────┴───────────────┴───────────────┘

---
> P-027: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [29]:
df_027 = (
    df_receipt
    .group_by("store_cd")
    .aggregate(mean_amount=df_receipt["amount"].mean())
    .order_by(ibis.desc("mean_amount"))
)

df_027.head(5)

┏━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ store_cd ┃ mean_amount ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ string   │ float64     │
├──────────┼─────────────┤
│ S13052   │  402.867470 │
│ S13015   │  351.111960 │
│ S13003   │  350.915519 │
│ S14010   │  348.791262 │
│ S13001   │  348.470386 │
└──────────┴─────────────┘

---
> P-028: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [30]:
df_028 = (
    df_receipt
    .group_by("store_cd")
    .aggregate(median_amount=df_receipt["amount"].median())
    .order_by(ibis.desc("median_amount"))
)

df_028.head(5)

┏━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ store_cd ┃ median_amount ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ string   │ float64       │
├──────────┼───────────────┤
│ S13052   │         190.0 │
│ S14010   │         188.0 │
│ S14050   │         185.0 │
│ S14040   │         180.0 │
│ S13018   │         180.0 │
└──────────┴───────────────┘

---
> P-029: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求め、10件表示させよ。

In [31]:
df_029 = (
    df_receipt
    .group_by("store_cd")
    .aggregate(
        mode_product_cd=df_receipt["product_cd"].mode(),
        count_product_cd=df_receipt["product_cd"].count()
    )
)

df_029.head(10)

┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ store_cd ┃ mode_product_cd ┃ count_product_cd ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ string   │ string          │ int64            │
├──────────┼─────────────────┼──────────────────┤
│ S14024   │ P060303001      │             2389 │
│ S14023   │ P071401001      │             2235 │
│ S13004   │ P060303001      │             2355 │
│ S12014   │ P060303001      │             2333 │
│ S14034   │ P060303001      │             2003 │
│ S13032   │ P060303001      │             2459 │
│ S12013   │ P060303001      │             2385 │
│ S13005   │ P040503001      │             1992 │
│ S13002   │ P060303001      │             2322 │
│ S13009   │ P060303001      │             2465 │
└──────────┴─────────────────┴──────────────────┘

---
> P-030: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の分散を計算し、降順で5件表示せよ。

In [32]:
df_030 = (
    df_receipt
    .group_by("store_cd")
    .aggregate(var_amount=df_receipt["amount"].var())
    .order_by(ibis.desc("var_amount"))
)

df_030.head(5)

┏━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ store_cd ┃ var_amount    ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ string   │ float64       │
├──────────┼───────────────┤
│ S13052   │ 441863.252526 │
│ S14011   │ 306442.242432 │
│ S14034   │ 297068.392740 │
│ S13001   │ 295558.842618 │
│ S13015   │ 295427.197086 │
└──────────┴───────────────┘

---
> P-031: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標準偏差を計算し、降順で5件表示せよ。

In [33]:
df_031 = (
    df_receipt
    .group_by("store_cd")
    .aggregate(std_amount=df_receipt["amount"].std())
    .order_by(ibis.desc("std_amount"))
)

df_031.head(5)

┏━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ store_cd ┃ std_amount ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━┩
│ string   │ float64    │
├──────────┼────────────┤
│ S13052   │ 664.727954 │
│ S14011   │ 553.572256 │
│ S14034   │ 545.039808 │
│ S13001   │ 543.653237 │
│ S13015   │ 543.532149 │
└──────────┴────────────┘

---
> P-032: レシート明細データ（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [34]:
for pct in range(0, 101, 25):
    print(f"{int(pct):<3} パーセンタイル:", df_receipt["amount"].execute().quantile(pct/100))

0   パーセンタイル: 10.0
25  パーセンタイル: 102.0
50  パーセンタイル: 170.0
75  パーセンタイル: 288.0
100 パーセンタイル: 10925.0


---
> P-033: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [35]:
df_033 = (
    df_receipt
    .group_by("store_cd")
    .aggregate(mean_amount=df_receipt["amount"].mean())
    .filter(_["mean_amount"] >= 330)
)
df_033

┏━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ store_cd ┃ mean_amount ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ string   │ float64     │
├──────────┼─────────────┤
│ S12013   │  330.194130 │
│ S13003   │  350.915519 │
│ S14026   │  332.340588 │
│ S13015   │  351.111960 │
│ S14045   │  330.082073 │
│ S14010   │  348.791262 │
│ S13001   │  348.470386 │
│ S14047   │  330.077073 │
│ S13004   │  330.943949 │
│ S13019   │  330.208616 │
│ …        │           … │
└──────────┴─────────────┘

---
> P-034: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [36]:
df_034 = (
    df_receipt
    .filter(~df_receipt["customer_id"].startswith("Z"))
    .group_by("customer_id")
    .aggregate(sum_amount=df_receipt["amount"].sum())
)

df_034["sum_amount"].mean().execute()

2547.742234529256

---
> P-035: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出し、10件表示せよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [37]:
df_035 = (
    df_receipt
    .filter(~df_receipt["customer_id"].startswith("Z"))
    .group_by("customer_id")
    .aggregate(sum_amount=df_receipt["amount"].sum())
    .filter(_["sum_amount"] >= _["sum_amount"].mean())
    .order_by("customer_id")
)

df_035.head(10)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ customer_id    ┃ sum_amount ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ string         │ int64      │
├────────────────┼────────────┤
│ CS001115000010 │       3044 │
│ CS001205000006 │       3337 │
│ CS001214000009 │       4685 │
│ CS001214000017 │       4132 │
│ CS001214000052 │       5639 │
│ CS001215000040 │       3496 │
│ CS001304000006 │       3726 │
│ CS001305000005 │       3485 │
│ CS001305000011 │       4370 │
│ CS001315000180 │       3300 │
└────────────────┴────────────┘

## DataFrameの結合

---
> P-036: レシート明細データ（df_receipt）と店舗データ（df_store）を内部結合し、レシート明細データの全項目と店舗データの店舗名（store_name）を10件表示せよ。

- 内部結合(inner):結合させる両方のDataFrameに存在する結合キー(on)で、一致する項目のみを抽出する結合方法
- 左外部結合(left):結合させる両方のDataFrameに存在する結合キー(on)で、基準となるDataFrameを左とする。基準DataFrameはすべての行が出力され、結合されたDataFrameは結合キーの値が基準の結合キーと一致した行のみ出力される
- 完全外部結合(outer):結合させる両方のDataFrameに存在する結合キー(on)で、基準となるDataFrameを左とする。基準のDataFrameおよび結合されたDataFrameのすべての行が出力される

In [38]:
df_036 = (
    df_receipt
    .join(
        df_store.select("store_cd", "store_name"),
        predicates="store_cd",
        how="inner"
    )
)

df_036.head(10)

┏━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┓
┃ sales_ymd ┃ sales_epoch ┃ store_cd ┃ receipt_no ┃ receipt_sub_no ┃ customer_id    ┃ product_cd ┃ quantity ┃ amount ┃ store_name ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━┩
│ int64     │ int64       │ string   │ int64      │ int64          │ string         │ string     │ int64    │ int64  │ string     │
├───────────┼─────────────┼──────────┼────────────┼────────────────┼────────────────┼────────────┼──────────┼────────┼────────────┤
│  20181103 │  1541203200 │ S14006   │        112 │              1 │ CS006214000001 │ P070305012 │        1 │    158 │ 葛が谷店   │
│  20181118 │  1542499200 │ S13008   │       1132 │              2 │ CS008415000097 │ P070701017 │        1 │     81 │ 成城店     │
│  20170712 │  1499817600 │ S14028   │       1102 │              1 │ CS028414000014 │ P060101005 │        1 │    170 │ 二ツ橋店   │
│  20190205 │  1549324800 │ S14042   │       1132 │              1 │ ZZ000000000000 │ P050301001 │        1 │     25 │ 新山下店   │
│  20180821 │  1534809600 │ S14025   │       1102 │              2 │ CS025415000050 │ P060102007 │        1 │     90 │ 大和店     │
│  20190605 │  1559692800 │ S13003   │       1112 │              1 │ CS003515000195 │ P050102002 │        1 │    138 │ 狛江店     │
│  20181205 │  1543968000 │ S14024   │       1102 │              2 │ CS024514000042 │ P080101005 │        1 │     30 │ 三田店     │
│  20190922 │  1569110400 │ S14040   │       1102 │              1 │ CS040415000178 │ P070501004 │        1 │    128 │ 長津田店   │
│  20170504 │  1493856000 │ S13020   │       1112 │              2 │ ZZ000000000000 │ P071302010 │        1 │    770 │ 十条仲原店 │
│  20191010 │  1570665600 │ S14027   │       1102 │              1 │ CS027514000015 │ P071101003 │        1 │    680 │ 南藤沢店   │
└───────────┴─────────────┴──────────┴────────────┴────────────────┴────────────────┴────────────┴──────────┴────────┴────────────┘

---
> P-037: 商品データ（df_product）とカテゴリデータ（df_category）を内部結合し、商品データの全項目とカテゴリデータのカテゴリ小区分名（category_small_name）を10件表示せよ。

In [39]:
df_037 = (
    df_product
    .join(
        df_category.select("category_small_cd", "category_small_name"),
        predicates="category_small_cd",
        how="inner"
    )
)

df_037.head(10)

┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ product_cd ┃ category_major_cd ┃ category_medium_cd ┃ category_small_cd ┃ unit_price ┃ unit_cost ┃ category_small_name ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ string     │ string            │ string             │ string            │ int64      │ int64     │ string              │
├────────────┼───────────────────┼────────────────────┼───────────────────┼────────────┼───────────┼─────────────────────┤
│ P040101001 │ 04                │ 0401               │ 040101            │        198 │       149 │ 弁当類              │
│ P040101002 │ 04                │ 0401               │ 040101            │        218 │       164 │ 弁当類              │
│ P040101003 │ 04                │ 0401               │ 040101            │        230 │       173 │ 弁当類              │
│ P040101004 │ 04                │ 0401               │ 040101            │        248 │       186 │ 弁当類              │
│ P040101005 │ 04                │ 0401               │ 040101            │        268 │       201 │ 弁当類              │
│ P040101006 │ 04                │ 0401               │ 040101            │        298 │       224 │ 弁当類              │
│ P040101007 │ 04                │ 0401               │ 040101            │        338 │       254 │ 弁当類              │
│ P040101008 │ 04                │ 0401               │ 040101            │        420 │       315 │ 弁当類              │
│ P040101009 │ 04                │ 0401               │ 040101            │        498 │       374 │ 弁当類              │
│ P040101010 │ 04                │ 0401               │ 040101            │        580 │       435 │ 弁当類              │
└────────────┴───────────────────┴────────────────────┴───────────────────┴────────────┴───────────┴─────────────────────┘

---
> P-038: 顧客データ（df_customer）とレシート明細データ（df_receipt）から、顧客ごとの売上金額合計を求め、10件表示せよ。ただし、売上実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [40]:
df_038 = (
    df_customer
    .join(df_receipt, predicates="customer_id", how="inner")
    .filter(
        (df_customer["gender_cd"] == "1")
        & (~df_customer["customer_id"].startswith("Z"))
    )
    .group_by("customer_id")
    .aggregate(_["amount"].sum().fill_null(0))
)

df_038.head(10)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ customer_id    ┃ Coalesce() ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ string         │ int64      │
├────────────────┼────────────┤
│ CS026515000042 │       6866 │
│ CS005513000061 │       2598 │
│ CS022414000065 │       4960 │
│ CS004413000039 │       2348 │
│ CS030415000197 │      11972 │
│ CS004515000723 │        809 │
│ CS008414000013 │       2523 │
│ CS032215000073 │       5520 │
│ CS031415000086 │       4535 │
│ CS040515000174 │       4301 │
└────────────────┴────────────┘

---
> P-039: レシート明細データ（df_receipt）から、売上日数の多い顧客の上位20件を抽出したデータと、売上金額合計の多い顧客の上位20件を抽出したデータをそれぞれ作成し、さらにその2つを完全外部結合せよ。ただし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [41]:
df_cnt = (
    df_receipt
    .filter(~df_receipt["customer_id"].startswith("Z"))
    .group_by("customer_id")
    .aggregate(sales_count=_["sales_ymd"].nunique())
    .order_by(ibis.desc("sales_count"))
    .head(20)
)

df_sum = (
    df_receipt
    .filter(~df_receipt["customer_id"].startswith("Z"))
    .group_by("customer_id")
    .aggregate(amount_sum=df_receipt["amount"].sum())
    .order_by(ibis.desc("amount_sum"))
    .head(20)
)

df_039 = (
    df_cnt.join(df_sum, how="outer", predicates="customer_id")
    .order_by(ibis.desc("sales_count"))
)

df_039

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ customer_id    ┃ sales_count ┃ customer_id_right ┃ amount_sum ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ string         │ int64       │ string            │ int64      │
├────────────────┼─────────────┼───────────────────┼────────────┤
│ CS040214000008 │          23 │ NULL              │       NULL │
│ CS015415000185 │          22 │ CS015415000185    │      20153 │
│ CS010214000010 │          22 │ CS010214000010    │      18585 │
│ CS028415000007 │          21 │ CS028415000007    │      19127 │
│ CS010214000002 │          21 │ NULL              │       NULL │
│ CS017415000097 │          20 │ CS017415000097    │      23086 │
│ CS016415000141 │          20 │ CS016415000141    │      18372 │
│ CS031414000051 │          19 │ CS031414000051    │      19202 │
│ CS022515000226 │          19 │ NULL              │       NULL │
│ CS039414000052 │          19 │ NULL              │       NULL │
│ …              │           … │ …                 │          … │
└────────────────┴─────────────┴───────────────────┴────────────┘

---
> P-040: 全ての店舗と全ての商品を組み合わせたデータを作成したい。店舗データ（df_store）と商品データ（df_product）を直積し、件数を計算せよ。

In [42]:
df_040 = (
    df_store.mutate(key=0)
    .join(df_product.mutate(key=0), how="outer", predicates="key")
)

df_040.count()

┌────────┐
│ 531590 │
└────────┘

---
> P-041: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前回売上があった日からの売上金額増減を計算せよ。そして結果を10件表示せよ。

In [43]:
df_041 = (
    df_receipt
    .group_by("sales_ymd")
    .aggregate(sum_amount=df_receipt["amount"].sum())
    .order_by("sales_ymd")
    .mutate(
        diff_amount=(_["sum_amount"] - _["sum_amount"].lag(1))
    )
)

df_041.head()

┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ sales_ymd ┃ sum_amount ┃ diff_amount ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ int64     │ int64      │ int64       │
├───────────┼────────────┼─────────────┤
│  20170101 │      33723 │        NULL │
│  20170102 │      24165 │       -9558 │
│  20170103 │      27503 │        3338 │
│  20170104 │      36165 │        8662 │
│  20170105 │      37830 │        1665 │
└───────────┴────────────┴─────────────┘

---
> P-042: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、前回、前々回、3回前に売上があった日のデータを結合せよ。そして結果を10件表示せよ。

In [44]:
df_042 = (
    df_receipt
    .group_by("sales_ymd")
    .aggregate(sum_amount=df_receipt["amount"].sum())
    .order_by("sales_ymd")
    .mutate(lag1=_["sum_amount"].lag(1))
    .mutate(lag2=_["sum_amount"].lag(2))
    .mutate(lag3=_["sum_amount"].lag(3))
    .drop_null()
)


df_042.head(10)

┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃ sales_ymd ┃ sum_amount ┃ lag1  ┃ lag2  ┃ lag3  ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ int64     │ int64      │ int64 │ int64 │ int64 │
├───────────┼────────────┼───────┼───────┼───────┤
│  20170104 │      36165 │ 27503 │ 24165 │ 33723 │
│  20170105 │      37830 │ 36165 │ 27503 │ 24165 │
│  20170106 │      32387 │ 37830 │ 36165 │ 27503 │
│  20170107 │      23415 │ 32387 │ 37830 │ 36165 │
│  20170108 │      24737 │ 23415 │ 32387 │ 37830 │
│  20170109 │      26718 │ 24737 │ 23415 │ 32387 │
│  20170110 │      20143 │ 26718 │ 24737 │ 23415 │
│  20170111 │      24287 │ 20143 │ 26718 │ 24737 │
│  20170112 │      23526 │ 24287 │ 20143 │ 26718 │
│  20170113 │      28004 │ 23526 │ 24287 │ 20143 │
└───────────┴────────────┴───────┴───────┴───────┘

---
> P-043： レシート明細データ（df_receipt）と顧客データ（df_customer）を結合し、性別コード（gender_cd）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータを作成せよ。性別コードは0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [45]:
import ibis.selectors as s
df_043 = (
    df_receipt
    .join(df_customer, predicates="customer_id", how="inner")
    .mutate(era=ibis._["age"] // 10 *10).cast({"era": str})
    .pivot_wider(
        id_cols="era",
        names_from="gender_cd",
        values_from="amount",
        values_agg="sum"
    )
    .rename({"female": "1", "male": "0", "unknown": "9"})
    .order_by("era")
)

df_043

┏━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┓
┃ era    ┃ unknown ┃ male   ┃ female  ┃
┡━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━┩
│ string │ int64   │ int64  │ int64   │
├────────┼─────────┼────────┼─────────┤
│ 10     │    4317 │   1591 │  149836 │
│ 20     │   44328 │  72940 │ 1363724 │
│ 30     │   50441 │ 177322 │  693047 │
│ 40     │  483512 │  19355 │ 9320791 │
│ 50     │  342923 │  54320 │ 6685192 │
│ 60     │   71418 │ 272469 │  987741 │
│ 70     │    2427 │  13435 │   29764 │
│ 80     │    5111 │  46360 │  262923 │
│ 90     │    NULL │   NULL │    6260 │
└────────┴─────────┴────────┴─────────┘

---
> P-044： 043で作成した売上サマリデータ（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を"00"、女性を"01"、不明を"99"とする。

In [46]:
df_044 = (
    df_043
    .rename({"01": "female", "00": "male", "99": "unknown"})
    .pivot_longer(
        # col=s.matches("(00)|(01)|(99)"),
        col=[s.c("00"), s.c("01"), s.c("99")],
        names_to="gender_cd",
        values_to="amount"
    )
    .order_by("era", "gender_cd")
)

df_044

┏━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┓
┃ era    ┃ gender_cd ┃ amount  ┃
┡━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━┩
│ string │ string    │ int64   │
├────────┼───────────┼─────────┤
│ 10     │ 00        │    1591 │
│ 10     │ 01        │  149836 │
│ 10     │ 99        │    4317 │
│ 20     │ 00        │   72940 │
│ 20     │ 01        │ 1363724 │
│ 20     │ 99        │   44328 │
│ 30     │ 00        │  177322 │
│ 30     │ 01        │  693047 │
│ 30     │ 99        │   50441 │
│ 40     │ 00        │   19355 │
│ …      │ …         │       … │
└────────┴───────────┴─────────┘

## Date型

---
> P-045: 顧客データ（df_customer）の生年月日（birth_day）は日付型でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [47]:
df_045 = (
    df_customer
    .mutate(birth_day=df_customer["birth_day"].strftime("%Y%m%d"))
    .select("customer_id", "birth_day")
)

df_045.head(10)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ customer_id    ┃ birth_day ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ string         │ string    │
├────────────────┼───────────┤
│ CS021313000114 │ 19810429  │
│ CS037613000071 │ 19520401  │
│ CS031415000172 │ 19761004  │
│ CS028811000001 │ 19330327  │
│ CS001215000145 │ 19950329  │
│ CS020401000016 │ 19740915  │
│ CS015414000103 │ 19770809  │
│ CS029403000008 │ 19730817  │
│ CS015804000004 │ 19310502  │
│ CS033513000180 │ 19620711  │
└────────────────┴───────────┘

---
> P-046: 顧客データ（df_customer）の申し込み日（application_date）はYYYYMMDD形式の文字列型でデータを保有している。これを日付型に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [48]:
df_046 = (
    df_customer
    .mutate(application_date=df_customer["application_date"].cast(str).to_date("%Y%m%d"))
    .select("customer_id", "application_date")
)

df_046.head(10)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ customer_id    ┃ application_date ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ string         │ date             │
├────────────────┼──────────────────┤
│ CS021313000114 │ 2015-09-05       │
│ CS037613000071 │ 2015-04-14       │
│ CS031415000172 │ 2015-05-29       │
│ CS028811000001 │ 2016-01-15       │
│ CS001215000145 │ 2017-06-05       │
│ CS020401000016 │ 2015-02-25       │
│ CS015414000103 │ 2015-07-22       │
│ CS029403000008 │ 2015-05-15       │
│ CS015804000004 │ 2015-06-07       │
│ CS033513000180 │ 2015-07-28       │
└────────────────┴──────────────────┘

---
> P-047: レシート明細データ（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型に変換し、レシート番号（receipt_no）、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [49]:
df_047 = (
    df_receipt
    .mutate(sales_ymd=df_receipt["sales_ymd"].cast(str).to_date("%Y%m%d"))
    .select(
        "receipt_no",
        "receipt_sub_no",
        "sales_ymd"
    )
)

df_047.head()

┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ receipt_no ┃ receipt_sub_no ┃ sales_ymd  ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ int64      │ int64          │ date       │
├────────────┼────────────────┼────────────┤
│        112 │              1 │ 2018-11-03 │
│       1132 │              2 │ 2018-11-18 │
│       1102 │              1 │ 2017-07-12 │
│       1132 │              1 │ 2019-02-05 │
│       1102 │              2 │ 2018-08-21 │
└────────────┴────────────────┴────────────┘

---
> P-048: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [50]:
# Polarsに変換しています

df_048 = (
    df_receipt
    .to_polars()
    .select(
        "receipt_no", "receipt_sub_no",
        pl.col("sales_epoch").cast(dtype=pl.Utf8).str.strptime(dtype=pl.Datetime, format="%s"),
    )
)

df_048.head(10)

receipt_no,receipt_sub_no,sales_epoch
i64,i64,datetime[μs]
112,1,2018-11-03 00:00:00
1132,2,2018-11-18 00:00:00
1102,1,2017-07-12 00:00:00
1132,1,2019-02-05 00:00:00
1102,2,2018-08-21 00:00:00
1112,1,2019-06-05 00:00:00
1102,2,2018-12-05 00:00:00
1102,1,2019-09-22 00:00:00
1112,2,2017-05-04 00:00:00


---
> P-049: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「年」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [51]:
# Polarsに変換しています

df_049 = df_receipt.to_polars().select(
    "receipt_no", "receipt_sub_no",
    pl.col("sales_epoch").cast(dtype=pl.Utf8).str.strptime(dtype=pl.Datetime, format="%s").dt.year(),
)


df_049.head(10)

receipt_no,receipt_sub_no,sales_epoch
i64,i64,i32
112,1,2018
1132,2,2018
1102,1,2017
1132,1,2019
1102,2,2018
1112,1,2019
1102,2,2018
1102,1,2019
1112,2,2017


---
> P-050: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「月」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「月」は0埋め2桁で取り出すこと。

In [52]:
# Polarsに変換しています

df_050 = df_receipt.to_polars().select(
    "receipt_no", "receipt_sub_no",
    # 0埋めなので文字列"strftime"で抽出
    pl.col("sales_epoch").cast(dtype=pl.Utf8).str.strptime(dtype=pl.Datetime, format="%s").dt.strftime("%m"),
)

df_050.head(10)

receipt_no,receipt_sub_no,sales_epoch
i64,i64,str
112,1,"""11"""
1132,2,"""11"""
1102,1,"""07"""
1132,1,"""02"""
1102,2,"""08"""
1112,1,"""06"""
1102,2,"""12"""
1102,1,"""09"""
1112,2,"""05"""


---
> P-051: レシート明細データ（df_receipt）の売上エポック秒を日付型に変換し、「日」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「日」は0埋め2桁で取り出すこと。

In [53]:
df_051 = df_receipt.to_polars().select(
    "receipt_no", "receipt_sub_no",
    pl.col("sales_epoch").cast(dtype=pl.Utf8).str.strptime(dtype=pl.Datetime, format="%s").dt.strftime("%d"),
)

df_051.head()

receipt_no,receipt_sub_no,sales_epoch
i64,i64,str
112,1,"""03"""
1132,2,"""18"""
1102,1,"""12"""
1132,1,"""05"""
1102,2,"""21"""


## 条件分岐

---
> P-052: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2,000円以下を0、2,000円より大きい金額を1に二値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [54]:
df_052 = (
    df_receipt
    .filter(~df_receipt["customer_id"].startswith("Z"))
    .group_by("customer_id")
    .aggregate(sum_amount=df_receipt["amount"].sum())
    .mutate(
        sales_flag=(
            ibis.case()
            .when(_["sum_amount"] <= 2000, 1)
            .else_(0)
            .end()
        )
    )
)

df_052.head(10)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ customer_id    ┃ sum_amount ┃ sales_flag ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ string         │ int64      │ int8       │
├────────────────┼────────────┼────────────┤
│ CS003515000195 │       5412 │          0 │
│ CS024514000042 │        533 │          1 │
│ CS026515000042 │       6866 │          0 │
│ CS005513000061 │       2598 │          0 │
│ CS022414000065 │       4960 │          0 │
│ CS004413000039 │       2348 │          0 │
│ CS030415000197 │      11972 │          0 │
│ CS004515000723 │        809 │          1 │
│ CS008414000013 │       2523 │          0 │
│ CS032215000073 │       5520 │          0 │
└────────────────┴────────────┴────────────┘

---
> P-053: 顧客データ（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に二値化せよ。さらにレシート明細データ（df_receipt）と結合し、全期間において売上実績のある顧客数を、作成した二値ごとにカウントせよ。

In [55]:
df_053 = (
    df_customer
    .mutate(
        postal_flag=(
            ibis.case()
            .when(df_customer["postal_cd"][:3].cast(int).between(100, 209), 1)
            .else_(0)
            .end()
        )
    )
    .join(df_receipt, predicates="customer_id", how="inner")
    .group_by("postal_flag")
    .aggregate(customer_id_n_unique=_["customer_id"].nunique())
)

df_053

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ postal_flag ┃ customer_id_n_unique ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ int8        │ int64                │
├─────────────┼──────────────────────┤
│           0 │                 3906 │
│           1 │                 4400 │
└─────────────┴──────────────────────┘

---
> P-054: 顧客データ（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに10件表示せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。

In [56]:
df_054 = (
    df_customer
    .select("customer_id", "address")
    .mutate(
        prefecture_cd=(
            ibis.case()
            .when(df_customer["address"].startswith("埼玉県"), "11")
            .when(df_customer["address"].startswith("千葉県"), "12")
            .when(df_customer["address"].startswith("東京都"), "13")
            .when(df_customer["address"].startswith("神奈川県"), "14")
            .else_("00")
            .end()
        )
    )
)

df_054.head(10)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ customer_id    ┃ address                ┃ prefecture_cd ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ string         │ string                 │ string        │
├────────────────┼────────────────────────┼───────────────┤
│ CS021313000114 │ 神奈川県伊勢原市粟窪*… │ 14            │
│ CS037613000071 │ 東京都江東区南砂*****… │ 13            │
│ CS031415000172 │ 東京都渋谷区代々木***… │ 13            │
│ CS028811000001 │ 神奈川県横浜市泉区和 … │ 14            │
│ CS001215000145 │ 東京都大田区仲六郷***… │ 13            │
│ CS020401000016 │ 東京都板橋区若木*****… │ 13            │
│ CS015414000103 │ 東京都江東区北砂*****… │ 13            │
│ CS029403000008 │ 千葉県浦安市海楽*****… │ 12            │
│ CS015804000004 │ 東京都江東区北砂*****… │ 13            │
│ CS033513000180 │ 神奈川県横浜市旭区善 … │ 14            │
└────────────────┴────────────────────────┴───────────────┘

---
> P-055: レシート明細（df_receipt）データの売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額合計とともに10件表示せよ。カテゴリ値は順に1〜4とする。
>
> - 最小値以上第1四分位未満 ・・・ 1を付与
> - 第1四分位以上第2四分位未満 ・・・ 2を付与
> - 第2四分位以上第3四分位未満 ・・・ 3を付与
> - 第3四分位以上 ・・・ 4を付与

In [57]:
df_055 = (
    df_receipt
    .group_by("customer_id")
    .aggregate(sum_amount=df_receipt["amount"].sum())
    .order_by("customer_id")
    .mutate(
        pct_group=(
            ibis.case()
            .when(_["sum_amount"] < _["sum_amount"].execute().quantile(0.25), 1)
            .when(_["sum_amount"] < _["sum_amount"].execute().quantile(0.50), 2)
            .when(_["sum_amount"] < _["sum_amount"].execute().quantile(0.75), 3)
            .else_(4)
            .end()
        )
    )
)

df_055.head(10)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ customer_id    ┃ sum_amount ┃ pct_group ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ string         │ int64      │ int8      │
├────────────────┼────────────┼───────────┤
│ CS001113000004 │       1298 │         2 │
│ CS001114000005 │        626 │         2 │
│ CS001115000010 │       3044 │         3 │
│ CS001205000004 │       1988 │         3 │
│ CS001205000006 │       3337 │         3 │
│ CS001211000025 │        456 │         1 │
│ CS001212000027 │        448 │         1 │
│ CS001212000031 │        296 │         1 │
│ CS001212000046 │        228 │         1 │
│ CS001212000070 │        456 │         1 │
└────────────────┴────────────┴───────────┘

---
> P-056: 顧客データ（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに10件表示せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。

In [58]:
df_056 = (
    df_customer
    .mutate(era=df_customer["age"] // 10 * 10)
    .select("customer_id", "birth_day", "era")
    .mutate(
        era=ibis.case()
        .when(_["era"] >= 60, 60)
        .else_(_["era"])
        .end()
    )
)

df_056.head(10)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ customer_id    ┃ birth_day  ┃ era   ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ string         │ date       │ int64 │
├────────────────┼────────────┼───────┤
│ CS021313000114 │ 1981-04-29 │    30 │
│ CS037613000071 │ 1952-04-01 │    60 │
│ CS031415000172 │ 1976-10-04 │    40 │
│ CS028811000001 │ 1933-03-27 │    60 │
│ CS001215000145 │ 1995-03-29 │    20 │
│ CS020401000016 │ 1974-09-15 │    40 │
│ CS015414000103 │ 1977-08-09 │    40 │
│ CS029403000008 │ 1973-08-17 │    40 │
│ CS015804000004 │ 1931-05-02 │    60 │
│ CS033513000180 │ 1962-07-11 │    50 │
└────────────────┴────────────┴───────┘

---
> P-057: 056の抽出結果と性別コード（gender_cd）により、新たに性別×年代の組み合わせを表すカテゴリデータを作成し、10件表示せよ。組み合わせを表すカテゴリの値は任意とする。

In [59]:
df_057 = (
    df_056.join(df_customer, predicates="customer_id", how="inner")
    .mutate(
        gender_era=_["gender_cd"] + _["era"].cast(str)
    )
    .select("customer_id", "birth_day", "era", "gender_era")
)

df_057.head(10)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━┓
┃ customer_id    ┃ birth_day  ┃ era   ┃ gender_era ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━┩
│ string         │ date       │ int64 │ string     │
├────────────────┼────────────┼───────┼────────────┤
│ CS021313000114 │ 1981-04-29 │    30 │ 130        │
│ CS037613000071 │ 1952-04-01 │    60 │ 960        │
│ CS031415000172 │ 1976-10-04 │    40 │ 140        │
│ CS028811000001 │ 1933-03-27 │    60 │ 160        │
│ CS001215000145 │ 1995-03-29 │    20 │ 120        │
│ CS020401000016 │ 1974-09-15 │    40 │ 040        │
│ CS015414000103 │ 1977-08-09 │    40 │ 140        │
│ CS029403000008 │ 1973-08-17 │    40 │ 040        │
│ CS015804000004 │ 1931-05-02 │    60 │ 060        │
│ CS033513000180 │ 1962-07-11 │    50 │ 150        │
└────────────────┴────────────┴───────┴────────────┘

---
> P-058: 顧客データ（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに10件表示せよ。

In [60]:
# できていない

## 統計処理

---
> P-059: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに10件表示せよ。標準化に使用する標準偏差は、分散の平方根、もしくは不偏分散の平方根のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

- 標準化
$$
x_{new}^i = \frac{x^i - \mu}{\sigma}
$$

In [61]:
df_059 = (
    df_receipt
    .filter(~df_receipt["customer_id"].startswith("Z"))
    .group_by("customer_id")
    .aggregate(sum_amount=df_receipt["amount"].sum())
    .mutate(
        stdardized_amount=(
            (_["sum_amount"] - _["sum_amount"].mean()) / (_["sum_amount"].std())
        )
    )
    .order_by("customer_id")
)

df_059.head(10)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ customer_id    ┃ sum_amount ┃ stdardized_amount ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ string         │ int64      │ float64           │
├────────────────┼────────────┼───────────────────┤
│ CS001113000004 │       1298 │         -0.459350 │
│ CS001114000005 │        626 │         -0.706348 │
│ CS001115000010 │       3044 │          0.182403 │
│ CS001205000004 │       1988 │         -0.205737 │
│ CS001205000006 │       3337 │          0.290096 │
│ CS001211000025 │        456 │         -0.768832 │
│ CS001212000027 │        448 │         -0.771773 │
│ CS001212000031 │        296 │         -0.827641 │
│ CS001212000046 │        228 │         -0.852635 │
│ CS001212000070 │        456 │         -0.768832 │
└────────────────┴────────────┴───────────────────┘

---
> P-060: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

- 正規化
$$
x_{new}^i = \frac{x^i - x_{min}}{x_{max} - x_{min}}
$$

In [62]:
df_060 = (
    df_receipt
    .filter(~df_receipt["customer_id"].startswith("Z"))
    .group_by("customer_id")
    .aggregate(sum_amount=df_receipt["amount"].sum())
    .mutate(
        normalized_amount=(
            (_["sum_amount"] - _["sum_amount"].min())
            / (_["sum_amount"].max() - _["sum_amount"].min())
        )
    )
    .order_by("customer_id")
)

df_060.head(10)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ customer_id    ┃ sum_amount ┃ normalized_amount ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ string         │ int64      │ float64           │
├────────────────┼────────────┼───────────────────┤
│ CS001113000004 │       1298 │          0.053354 │
│ CS001114000005 │        626 │          0.024157 │
│ CS001115000010 │       3044 │          0.129214 │
│ CS001205000004 │       1988 │          0.083333 │
│ CS001205000006 │       3337 │          0.141945 │
│ CS001211000025 │        456 │          0.016771 │
│ CS001212000027 │        448 │          0.016423 │
│ CS001212000031 │        296 │          0.009819 │
│ CS001212000046 │        228 │          0.006865 │
│ CS001212000070 │        456 │          0.016771 │
└────────────────┴────────────┴───────────────────┘

---
> P-061: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を常用対数化（底10）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [63]:
df_061 = (
    df_receipt
    .filter(~df_receipt["customer_id"].startswith("Z"))
    .group_by("customer_id")
    .aggregate(sum_amount=df_receipt["amount"].sum())
    .mutate(log10_amount=_["sum_amount"].log10())
    .order_by("customer_id")
)

df_061.head(10)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ customer_id    ┃ sum_amount ┃ log10_amount ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ string         │ int64      │ float64      │
├────────────────┼────────────┼──────────────┤
│ CS001113000004 │       1298 │     3.113275 │
│ CS001114000005 │        626 │     2.796574 │
│ CS001115000010 │       3044 │     3.483445 │
│ CS001205000004 │       1988 │     3.298416 │
│ CS001205000006 │       3337 │     3.523356 │
│ CS001211000025 │        456 │     2.658965 │
│ CS001212000027 │        448 │     2.651278 │
│ CS001212000031 │        296 │     2.471292 │
│ CS001212000046 │        228 │     2.357935 │
│ CS001212000070 │        456 │     2.658965 │
└────────────────┴────────────┴──────────────┘

---
> P-062: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を自然対数化（底e）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [64]:
df_062 = (
    df_receipt
    .filter(~df_receipt["customer_id"].startswith("Z"))
    .group_by("customer_id")
    .aggregate(sum_amount=df_receipt["amount"].sum())
    .mutate(log10_amount=_["sum_amount"].ln())
    .order_by("customer_id")
)

df_062.head(10)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ customer_id    ┃ sum_amount ┃ log10_amount ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ string         │ int64      │ float64      │
├────────────────┼────────────┼──────────────┤
│ CS001113000004 │       1298 │     7.168580 │
│ CS001114000005 │        626 │     6.439350 │
│ CS001115000010 │       3044 │     8.020928 │
│ CS001205000004 │       1988 │     7.594884 │
│ CS001205000006 │       3337 │     8.112827 │
│ CS001211000025 │        456 │     6.122493 │
│ CS001212000027 │        448 │     6.104793 │
│ CS001212000031 │        296 │     5.690359 │
│ CS001212000046 │        228 │     5.429346 │
│ CS001212000070 │        456 │     6.122493 │
└────────────────┴────────────┴──────────────┘

---
> P-063: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から各商品の利益額を算出し、結果を10件表示せよ。

In [65]:
df_063 = (
    df_product
    .mutate(unit_profit=(df_product["unit_price"] - df_product["unit_cost"]))
    .select("product_cd", "unit_price", "unit_cost", "unit_profit")
)

df_063.head(10)

┏━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ product_cd ┃ unit_price ┃ unit_cost ┃ unit_profit ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ string     │ int64      │ int64     │ int64       │
├────────────┼────────────┼───────────┼─────────────┤
│ P040101001 │        198 │       149 │          49 │
│ P040101002 │        218 │       164 │          54 │
│ P040101003 │        230 │       173 │          57 │
│ P040101004 │        248 │       186 │          62 │
│ P040101005 │        268 │       201 │          67 │
│ P040101006 │        298 │       224 │          74 │
│ P040101007 │        338 │       254 │          84 │
│ P040101008 │        420 │       315 │         105 │
│ P040101009 │        498 │       374 │         124 │
│ P040101010 │        580 │       435 │         145 │
└────────────┴────────────┴───────────┴─────────────┘

---
> P-064: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。ただし、単価と原価には欠損が生じていることに注意せよ。

In [66]:
df_064 = (
    df_product
    .mutate(unit_profit_rate=(df_product["unit_price"] - df_product["unit_cost"])/(df_product["unit_price"]))
)

df_064["unit_profit_rate"].mean().execute()

0.24911389885176904

---
> P-065: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [67]:
df_065 = (
    df_product
    .filter((~df_product["unit_price"].isnull()) & (~df_product["unit_cost"].isnull()))
    .select("product_cd", "unit_price", "unit_cost")
    .mutate(new_price=(_["unit_cost"] / 0.7).floor().cast(int))
    .mutate(new_profit_rate=(
        (_["new_price"] - _["unit_cost"])/_["new_price"]
    ))
    .order_by("product_cd")
)

df_065.head(10)

┏━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ product_cd ┃ unit_price ┃ unit_cost ┃ new_price ┃ new_profit_rate ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ string     │ int64      │ int64     │ int64     │ float64         │
├────────────┼────────────┼───────────┼───────────┼─────────────────┤
│ P040101001 │        198 │       149 │       212 │        0.297170 │
│ P040101002 │        218 │       164 │       234 │        0.299145 │
│ P040101003 │        230 │       173 │       247 │        0.299595 │
│ P040101004 │        248 │       186 │       265 │        0.298113 │
│ P040101005 │        268 │       201 │       287 │        0.299652 │
│ P040101006 │        298 │       224 │       320 │        0.300000 │
│ P040101007 │        338 │       254 │       362 │        0.298343 │
│ P040101008 │        420 │       315 │       450 │        0.300000 │
│ P040101009 │        498 │       374 │       534 │        0.299625 │
│ P040101010 │        580 │       435 │       621 │        0.299517 │
└────────────┴────────────┴───────────┴───────────┴─────────────────┘

---
> P-066: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を丸めること（四捨五入または偶数への丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [68]:
df_066 = (
    df_product
    .filter((~df_product["unit_price"].isnull()) & (~df_product["unit_cost"].isnull()))
    .select("product_cd", "unit_price", "unit_cost")
    .mutate(new_price=(_["unit_cost"] / 0.7).round().cast(int))
    .mutate(new_profit_rate=(
        (_["new_price"] - _["unit_cost"])/_["new_price"]
    ))
    .order_by("product_cd")
)

df_066.head()

┏━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ product_cd ┃ unit_price ┃ unit_cost ┃ new_price ┃ new_profit_rate ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ string     │ int64      │ int64     │ int64     │ float64         │
├────────────┼────────────┼───────────┼───────────┼─────────────────┤
│ P040101001 │        198 │       149 │       213 │        0.300469 │
│ P040101002 │        218 │       164 │       234 │        0.299145 │
│ P040101003 │        230 │       173 │       247 │        0.299595 │
│ P040101004 │        248 │       186 │       266 │        0.300752 │
│ P040101005 │        268 │       201 │       287 │        0.299652 │
└────────────┴────────────┴───────────┴───────────┴─────────────────┘

---
> P-067: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [69]:
df_067 = (
    df_product
    .filter((~df_product["unit_price"].isnull()) & (~df_product["unit_cost"].isnull()))
    .select("product_cd", "unit_price", "unit_cost")
    .mutate(new_price=(_["unit_cost"] / 0.7).ceil().cast(int))
    .mutate(new_profit_rate=(
        (_["new_price"] - _["unit_cost"])/_["new_price"]
    ))
    .order_by("product_cd")
)

df_067.head()

┏━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ product_cd ┃ unit_price ┃ unit_cost ┃ new_price ┃ new_profit_rate ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ string     │ int64      │ int64     │ int64     │ float64         │
├────────────┼────────────┼───────────┼───────────┼─────────────────┤
│ P040101001 │        198 │       149 │       213 │        0.300469 │
│ P040101002 │        218 │       164 │       235 │        0.302128 │
│ P040101003 │        230 │       173 │       248 │        0.302419 │
│ P040101004 │        248 │       186 │       266 │        0.300752 │
│ P040101005 │        268 │       201 │       288 │        0.302083 │
└────────────┴────────────┴───────────┴───────────┴─────────────────┘

---
> P-068: 商品データ（df_product）の各商品について、消費税率10％の税込み金額を求めよ。1円未満の端数は切り捨てとし、結果を10件表示せよ。ただし、単価（unit_price）には欠損が生じていることに注意せよ。

In [70]:
df_068 = (
    df_product
    .filter(~df_product["unit_price"].isnull())
    .select("product_cd", "unit_price")
    .mutate(tax_price=(_["unit_price"] * 1.1).cast(int))
    .order_by("product_cd")
)

df_068.head(10)

┏━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ product_cd ┃ unit_price ┃ tax_price ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ string     │ int64      │ int64     │
├────────────┼────────────┼───────────┤
│ P040101001 │        198 │       218 │
│ P040101002 │        218 │       240 │
│ P040101003 │        230 │       253 │
│ P040101004 │        248 │       273 │
│ P040101005 │        268 │       295 │
│ P040101006 │        298 │       328 │
│ P040101007 │        338 │       372 │
│ P040101008 │        420 │       462 │
│ P040101009 │        498 │       548 │
│ P040101010 │        580 │       638 │
└────────────┴────────────┴───────────┘

---
> P-069: レシート明細データ（df_receipt）と商品データ（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分コード（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分コード"07"（瓶詰缶詰）の売上実績がある顧客のみとし、結果を10件表示せよ。

In [71]:
df_069 = (
    df_receipt
    .join(df_product, predicates="product_cd", how="inner")
    .mutate(category_07_sell=(
        ibis.case()
        .when(
            _["category_major_cd"].startswith("07"),
            _["unit_price"] * _["quantity"]
        )
        .end()
    ))
    .group_by("customer_id")
    .aggregate(sum_all=_["amount"].sum(), sum_07=_["category_07_sell"].sum())
    .filter(~_["sum_07"].isnull())
    .mutate(sales_rate=(_["sum_07"] / _["sum_all"]))
    .order_by("customer_id")
)

df_069.head(10)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┓
┃ customer_id    ┃ sum_all ┃ sum_07 ┃ sales_rate ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━┩
│ string         │ int64   │ int64  │ float64    │
├────────────────┼─────────┼────────┼────────────┤
│ CS001113000004 │    1298 │   1298 │   1.000000 │
│ CS001114000005 │     626 │    486 │   0.776358 │
│ CS001115000010 │    3044 │   2694 │   0.885020 │
│ CS001205000004 │    1988 │    346 │   0.174044 │
│ CS001205000006 │    3337 │   2004 │   0.600539 │
│ CS001212000027 │     448 │    200 │   0.446429 │
│ CS001212000031 │     296 │    296 │   1.000000 │
│ CS001212000046 │     228 │    108 │   0.473684 │
│ CS001212000070 │     456 │    308 │   0.675439 │
│ CS001213000018 │     243 │    145 │   0.596708 │
└────────────────┴─────────┴────────┴────────────┘

## Date型の計算

---
> P-070: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [72]:
df_070 = (
    df_receipt
    .select("customer_id", "sales_ymd")
    .join(
        df_customer.select("customer_id", "application_date"),
        predicates="customer_id",
        how="inner"
    )
    .mutate(
        sales_ymd=_["sales_ymd"].cast(str).to_date("%Y%m%d"),
        application_date=_["application_date"].cast(str).to_date("%Y%m%d"),
    )
    .mutate(
        elapsed_date=_["sales_ymd"].delta(_["application_date"], "day")
    )
    .order_by("customer_id")
)

df_070.head(10)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ customer_id    ┃ sales_ymd  ┃ application_date ┃ elapsed_date ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ string         │ date       │ date             │ int64        │
├────────────────┼────────────┼──────────────────┼──────────────┤
│ CS001113000004 │ 2019-03-08 │ 2015-11-05       │         1219 │
│ CS001113000004 │ 2019-03-08 │ 2015-11-05       │         1219 │
│ CS001114000005 │ 2018-05-03 │ 2016-04-12       │          751 │
│ CS001114000005 │ 2019-07-31 │ 2016-04-12       │         1205 │
│ CS001114000005 │ 2019-07-31 │ 2016-04-12       │         1205 │
│ CS001114000005 │ 2018-05-03 │ 2016-04-12       │          751 │
│ CS001115000010 │ 2018-07-01 │ 2015-04-17       │         1171 │
│ CS001115000010 │ 2019-04-05 │ 2015-04-17       │         1449 │
│ CS001115000010 │ 2017-12-28 │ 2015-04-17       │          986 │
│ CS001115000010 │ 2018-07-01 │ 2015-04-17       │         1171 │
└────────────────┴────────────┴──────────────────┴──────────────┘

---
> P-071: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [73]:
df_071 = (
    df_receipt
    .select("customer_id", "sales_ymd")
    .join(
        df_customer.select("customer_id", "application_date"),
        predicates="customer_id",
        how="inner"
    )
    .mutate(
        sales_ymd=_["sales_ymd"].cast(str).to_date("%Y%m%d"),
        application_date=_["application_date"].cast(str).to_date("%Y%m%d"),
    )
    .mutate(
        elapsed_month=_["sales_ymd"].delta(_["application_date"], "month")
    )
    .order_by("customer_id")
)

df_071.head(10)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ customer_id    ┃ sales_ymd  ┃ application_date ┃ elapsed_month ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ string         │ date       │ date             │ int64         │
├────────────────┼────────────┼──────────────────┼───────────────┤
│ CS001113000004 │ 2019-03-08 │ 2015-11-05       │            40 │
│ CS001113000004 │ 2019-03-08 │ 2015-11-05       │            40 │
│ CS001114000005 │ 2018-05-03 │ 2016-04-12       │            25 │
│ CS001114000005 │ 2019-07-31 │ 2016-04-12       │            39 │
│ CS001114000005 │ 2019-07-31 │ 2016-04-12       │            39 │
│ CS001114000005 │ 2018-05-03 │ 2016-04-12       │            25 │
│ CS001115000010 │ 2018-07-01 │ 2015-04-17       │            39 │
│ CS001115000010 │ 2019-04-05 │ 2015-04-17       │            48 │
│ CS001115000010 │ 2017-12-28 │ 2015-04-17       │            32 │
│ CS001115000010 │ 2018-07-01 │ 2015-04-17       │            39 │
└────────────────┴────────────┴──────────────────┴───────────────┘

---
> P-072: レシート明細データ（df_receipt）の売上日（df_customer）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [74]:
df_072 = (
    df_receipt
    .select("customer_id", "sales_ymd")
    .join(
        df_customer.select("customer_id", "application_date"),
        predicates="customer_id",
        how="inner"
    )
    .mutate(
        sales_ymd=_["sales_ymd"].cast(str).to_date("%Y%m%d"),
        application_date=_["application_date"].cast(str).to_date("%Y%m%d"),
    )
    .mutate(
        elapsed_year=_["sales_ymd"].delta(_["application_date"], "year")
    )
)

df_072.head(10)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ customer_id    ┃ sales_ymd  ┃ application_date ┃ elapsed_year ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ string         │ date       │ date             │ int64        │
├────────────────┼────────────┼──────────────────┼──────────────┤
│ CS006214000001 │ 2018-11-03 │ 2015-02-01       │            3 │
│ CS008415000097 │ 2018-11-18 │ 2015-03-22       │            3 │
│ CS028414000014 │ 2017-07-12 │ 2015-07-11       │            2 │
│ CS025415000050 │ 2018-08-21 │ 2016-01-31       │            2 │
│ CS040415000178 │ 2019-09-22 │ 2015-06-27       │            4 │
│ CS025415000134 │ 2019-09-18 │ 2015-07-20       │            4 │
│ CS021515000126 │ 2017-10-10 │ 2015-05-08       │            2 │
│ CS039414000052 │ 2018-05-06 │ 2015-05-09       │            3 │
│ CS016215000032 │ 2019-03-26 │ 2015-07-02       │            4 │
│ CS026515000042 │ 2019-06-03 │ 2015-07-21       │            4 │
└────────────────┴────────────┴──────────────────┴──────────────┘

---
> P-073: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [75]:
df_073 = (
    df_receipt
    .select("customer_id", "sales_ymd")
    .join(
        df_customer.select("customer_id", "application_date"),
        predicates="customer_id",
        how="inner"
    )
    .mutate(
        sales_ymd=_["sales_ymd"].cast(str).to_date("%Y%m%d"),
        application_date=_["application_date"].cast(str).to_date("%Y%m%d"),
    )
    .mutate(
        elapsed_epoch=_["sales_ymd"].delta(_["application_date"], "second")
    )
    .order_by("customer_id")
)

df_073.head()

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ customer_id    ┃ sales_ymd  ┃ application_date ┃ elapsed_epoch ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ string         │ date       │ date             │ int64         │
├────────────────┼────────────┼──────────────────┼───────────────┤
│ CS001113000004 │ 2019-03-08 │ 2015-11-05       │     105321600 │
│ CS001113000004 │ 2019-03-08 │ 2015-11-05       │     105321600 │
│ CS001114000005 │ 2018-05-03 │ 2016-04-12       │      64886400 │
│ CS001114000005 │ 2018-05-03 │ 2016-04-12       │      64886400 │
│ CS001114000005 │ 2019-07-31 │ 2016-04-12       │     104112000 │
└────────────────┴────────────┴──────────────────┴───────────────┘

---
> P-074: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、直前の月曜日付とともに10件表示せよ（sales_ymdは数値でデータを保持している点に注意）。

In [76]:
#

## ランダム抽出

---
> P-075:顧客データ（df_customer）からランダムに1%のデータを抽出し、先頭から10件表示せよ。

In [77]:
df_075 = df_customer.sample(fraction=0.01)  # ランダム抽出

df_075.head(10)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ customer_id    ┃ customer_name ┃ gender_cd ┃ gender ┃ birth_day  ┃ age   ┃ postal_cd ┃ address                 ┃ application_store_cd ┃ application_date ┃ status_cd    ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ string         │ string        │ string    │ string │ date       │ int64 │ string    │ string                  │ string               │ int64            │ string       │
├────────────────┼───────────────┼───────────┼────────┼────────────┼───────┼───────────┼─────────────────────────┼──────────────────────┼──────────────────┼──────────────┤
│ CS009514000047 │ 三好 恭子     │ 1         │ 女性   │ 1960-01-21 │    59 │ 158-0095  │ 東京都世田谷区瀬田****… │ S13009               │         20150615 │ B-20100208-A │
│ CS017712000026 │ 長野 明日     │ 1         │ 女性   │ 1946-04-20 │    72 │ 166-0011  │ 東京都杉並区梅里******… │ S13017               │         20150213 │ 0-00000000-0 │
│ CS003414000214 │ 野口 恵梨香   │ 1         │ 女性   │ 1969-03-23 │    50 │ 201-0001  │ 東京都狛江市西野川****… │ S13003               │         20160630 │ E-20101007-B │
│ CS001703000031 │ 佐川 草太     │ 0         │ 男性   │ 1946-05-31 │    72 │ 144-0035  │ 東京都大田区南蒲田****… │ S13001               │         20160714 │ 0-00000000-0 │
│ CS034312000090 │ 江崎 景子     │ 1         │ 女性   │ 1987-01-24 │    32 │ 213-0005  │ 神奈川県川崎市高津区北… │ S14034               │         20151006 │ 0-00000000-0 │
│ CS004415000714 │ 杉本 愛梨     │ 1         │ 女性   │ 1977-01-13 │    42 │ 165-0033  │ 東京都中野区若宮******… │ S13004               │         20170916 │ 2-20100122-1 │
│ CS001314000201 │ 吉井 なぎさ   │ 1         │ 女性   │ 1983-11-23 │    35 │ 144-0055  │ 東京都大田区仲六郷****… │ S13001               │         20170510 │ 0-00000000-0 │
│ CS046611000001 │ 浅田 さとみ   │ 1         │ 女性   │ 1955-07-09 │    63 │ 223-0066  │ 神奈川県横浜市港北区高… │ S14046               │         20170620 │ 0-00000000-0 │
│ CS003513000408 │ 大久保 麻緒   │ 1         │ 女性   │ 1967-10-15 │    51 │ 182-0007  │ 東京都調布市菊野台****… │ S13003               │         20160303 │ 7-20091123-2 │
│ CS015502000055 │ 玉木 竜也     │ 0         │ 男性   │ 1967-01-02 │    52 │ 135-0048  │ 東京都江東区門前仲町**… │ S13015               │         20151012 │ 0-00000000-0 │
└────────────────┴───────────────┴───────────┴────────┴────────────┴───────┴───────────┴─────────────────────────┴──────────────────────┴──────────────────┴──────────────┘

---
> P-076: 顧客データ（df_customer）から性別コード（gender_cd）の割合に基づきランダムに10%のデータを層化抽出し、性別コードごとに件数を集計せよ。

In [78]:
#

## 外れ値と欠損値

---
> P-077: レシート明細データ（df_receipt）の売上金額を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。なお、外れ値は売上金額合計を対数化したうえで平均と標準偏差を計算し、その平均から3σを超えて離れたものとする（自然対数と常用対数のどちらでも可）。結果は10件表示せよ。

In [79]:
df_077 = (
    df_receipt
    .group_by("customer_id")
    .aggregate(sum_amount=_["amount"].sum())
    .mutate(log_amount=_["sum_amount"].ln())
    .filter(
        (
            (_["log_amount"] - _["log_amount"].mean()).abs() > (_["log_amount"].std() * 3)
        )
    )
)

df_077.head(10)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ customer_id    ┃ sum_amount ┃ log_amount ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ string         │ int64      │ float64    │
├────────────────┼────────────┼────────────┤
│ ZZ000000000000 │   12395003 │  16.332804 │
└────────────────┴────────────┴────────────┘

---
> P-078: レシート明細データ（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第1四分位と第3四分位の差であるIQRを用いて、「第1四分位数-1.5×IQR」を下回るもの、または「第3四分位数+1.5×IQR」を超えるものとする。結果は10件表示せよ。

In [80]:
df_078 = (
    df_receipt
    .group_by("customer_id")
    .aggregate(sum_amount=_["amount"].sum())
    .filter(
        (~_["customer_id"].startswith("Z"))
        & (_["sum_amount"] < _["sum_amount"].quantile(0.25) - 1.5 * (_["sum_amount"].quantile(0.75) - _["sum_amount"].quantile(0.25)))
        | (_["sum_amount"] > _["sum_amount"].quantile(0.25) + 1.5 * (_["sum_amount"].quantile(0.75) + _["sum_amount"].quantile(0.25)))
    )
    .order_by("customer_id")
)

df_078.head(10)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ customer_id    ┃ sum_amount ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ string         │ int64      │
├────────────────┼────────────┤
│ CS001414000048 │       8584 │
│ CS001605000009 │      18925 │
│ CS002214000043 │       8073 │
│ CS002415000594 │       9568 │
│ CS002415000728 │       7658 │
│ CS002515000032 │       7468 │
│ CS003414000017 │       7214 │
│ CS003515000070 │       7023 │
│ CS004414000181 │       9584 │
│ CS004415000106 │       7076 │
└────────────────┴────────────┘

---
> P-079: 商品データ（df_product）の各項目に対し、欠損数を確認せよ。

In [81]:
df_product

┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ product_cd ┃ category_major_cd ┃ category_medium_cd ┃ category_small_cd ┃ unit_price ┃ unit_cost ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ string     │ string            │ string             │ string            │ int64      │ int64     │
├────────────┼───────────────────┼────────────────────┼───────────────────┼────────────┼───────────┤
│ P040101001 │ 04                │ 0401               │ 040101            │        198 │       149 │
│ P040101002 │ 04                │ 0401               │ 040101            │        218 │       164 │
│ P040101003 │ 04                │ 0401               │ 040101            │        230 │       173 │
│ P040101004 │ 04                │ 0401               │ 040101            │        248 │       186 │
│ P040101005 │ 04                │ 0401               │ 040101            │        268 │       201 │
│ P040101006 │ 04                │ 0401               │ 040101            │        298 │       224 │
│ P040101007 │ 04                │ 0401               │ 040101            │        338 │       254 │
│ P040101008 │ 04                │ 0401               │ 040101            │        420 │       315 │
│ P040101009 │ 04                │ 0401               │ 040101            │        498 │       374 │
│ P040101010 │ 04                │ 0401               │ 040101            │        580 │       435 │
│ …          │ …                 │ …                  │ …                 │          … │         … │
└────────────┴───────────────────┴────────────────────┴───────────────────┴────────────┴───────────┘

In [82]:
df_product.sql(
    f"""
    SELECT COUNT(CASE WHEN unit_price IS NULL THEN 1 END) As null_count
    FROM {df_product.get_name()}
    """
).execute().to_numpy()[0][0]

7

In [83]:
df_product.get_name()

'ibis_read_csv_rm5zornt3naw7pbnpa63sdzr4m'

In [84]:
for col in df_product.columns:
    count = df_product.sql(
        f"""
        SELECT COUNT(CASE WHEN {col} IS NULL THEN 1 END) As null_count
        FROM {df_product.get_name()}
        """
    ).execute().to_numpy()[0][0]
    print(f"{col}\tNullの数:", count)

product_cd	Nullの数: 0
category_major_cd	Nullの数: 0
category_medium_cd	Nullの数: 0
category_small_cd	Nullの数: 0
unit_price	Nullの数: 7
unit_cost	Nullの数: 7


In [85]:
df_product.to_pandas().isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            7
unit_cost             7
dtype: int64

---
> P-080: 商品データ（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たな商品データを作成せよ。なお、削除前後の件数を表示させ、079で確認した件数だけ減少していることも確認すること。

In [86]:
print("削除前:", df_product.count().execute())
print("削除後:", df_product.drop_null().count().execute())

削除前: 10030
削除後: 10023


---
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たな商品データを作成せよ。なお、平均値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [87]:
df_081 = (
    df_product
    .mutate(unit_cost=df_product["unit_cost"].cast(float),
            unit_price=df_product["unit_price"].cast(float))
    .fill_null(
        {
            "unit_cost": int(df_product["unit_cost"].mean().execute()),
            "unit_price": int(df_product["unit_price"].mean().execute())
        }
    )
)

df_081.to_pandas().isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64

---
> P-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [88]:
df_082 = (
    df_product
    .mutate(unit_cost=df_product["unit_cost"].cast(float),
            unit_price=df_product["unit_price"].cast(float))
    .fill_null(
        {
            "unit_cost": int(df_product["unit_cost"].median().execute()),
            "unit_price": int(df_product["unit_price"].median().execute())
        }
    )
)

df_082.to_pandas().isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64

---
> P-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品のカテゴリ小区分コード（category_small_cd）ごとに算出した中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [89]:
df_083 = (
    df_product
    .group_by("category_small_cd")
    .aggregate(
        median_price=_["unit_price"].median(),
        median_cost=_["unit_cost"].median()
    )
    .join(df_product, how="inner", predicates="category_small_cd")
    .mutate(
        unit_price=(
            ibis.case()
            .when(_["unit_price"].isnull(), _["median_price"])
            .else_(_["unit_price"])
            .end()
        ),
        unit_cost=(
            ibis.case()
            .when(_["unit_cost"].isnull(), _["median_cost"])
            .else_(_["unit_cost"])
            .end()
        )
    )
)

df_083.to_pandas().isnull().sum()

category_small_cd     0
median_price          0
median_cost           0
product_cd            0
category_major_cd     0
category_medium_cd    0
unit_price            0
unit_cost             0
dtype: int64

## 応用

---
> P-084: 顧客データ（df_customer）の全顧客に対して全期間の売上金額に占める2019年売上金額の割合を計算し、新たなデータを作成せよ。ただし、売上実績がない場合は0として扱うこと。そして計算した割合が0超のものを抽出し、結果を10件表示せよ。また、作成したデータに欠損が存在しないことを確認せよ。

In [90]:
df_084 = (
    df_receipt
    .filter((20190101 <= df_receipt["sales_ymd"]) & (df_receipt["sales_ymd"] <= 20191231))
    .group_by("customer_id")
    .aggregate(amount_2019=_["amount"].sum())
    .join(
        df_receipt.group_by("customer_id").aggregate(amount_all=df_receipt["amount"].sum()),
        how="inner", predicates="customer_id"
    )
    .mutate(
        amount_2019=_["amount_2019"].fill_null(0).name("amount_2019"),
        amount_all=_["amount_all"].fill_null(0).name("amount_all"),
        amount_rate=(
            ibis.case()
            .when(_["amount_2019"] == 0, 0)
            .else_(_["amount_2019"]/_["amount_all"])
            .end()
        )
    )
    .filter(_["amount_rate"] > 0)
)

df_084.to_pandas().isnull().sum()

customer_id    0
amount_2019    0
amount_all     0
amount_rate    0
dtype: int64

---
> P-085: 顧客データ（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いてジオコードデータ（df_geocode）を紐付け、新たな顧客データを作成せよ。ただし、1つの郵便番号（postal_cd）に複数の経度（longitude）、緯度（latitude）情報が紐づく場合は、経度（longitude）、緯度（latitude）の平均値を算出して使用すること。また、作成結果を確認するために結果を10件表示せよ。

In [91]:
df_085 = df_customer.join(
    df_geocode.group_by("postal_cd").aggregate(
        longitude=_["longitude"].mean(),
        latitude=_["latitude"].mean()
    ),
    how="inner", predicates="postal_cd"
).order_by("customer_id")

df_085.head(10)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┓
┃ customer_id    ┃ customer_name ┃ gender_cd ┃ gender ┃ birth_day  ┃ age   ┃ postal_cd ┃ address             ┃ application_store_cd ┃ application_date ┃ status_cd    ┃ longitude ┃ latitude ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━┩
│ string         │ string        │ string    │ string │ date       │ int64 │ string    │ string              │ string               │ int64            │ string       │ float64   │ float64  │
├────────────────┼───────────────┼───────────┼────────┼────────────┼───────┼───────────┼─────────────────────┼──────────────────────┼──────────────────┼──────────────┼───────────┼──────────┤
│ CS001105000001 │ 中島 利夫     │ 0         │ 男性   │ 2000-01-14 │    19 │ 144-0056  │ 東京都大田区西六郷… │ S13001               │         20170310 │ 0-00000000-0 │ 139.70238 │ 35.54137 │
│ CS001112000009 │ 秦 美里       │ 1         │ 女性   │ 2006-08-24 │    12 │ 143-0026  │ 東京都大田区西馬込… │ S13001               │         20150703 │ 0-00000000-0 │ 139.70386 │ 35.58670 │
│ CS001112000019 │ 門脇 莉沙     │ 1         │ 女性   │ 2001-01-31 │    18 │ 143-0004  │ 東京都大田区昭和島… │ S13001               │         20170207 │ 0-00000000-0 │ 139.74687 │ 35.57153 │
│ CS001112000021 │ 長澤 麗奈     │ 1         │ 女性   │ 2001-12-15 │    17 │ 144-0056  │ 東京都大田区西六郷… │ S13001               │         20170612 │ 0-00000000-0 │ 139.70238 │ 35.54137 │
│ CS001112000023 │ 戸田 一恵     │ 1         │ 女性   │ 2004-01-26 │    15 │ 143-0004  │ 東京都大田区昭和島… │ S13001               │         20170724 │ 0-00000000-0 │ 139.74687 │ 35.57153 │
│ CS001112000024 │ 菊地 美幸     │ 1         │ 女性   │ 2001-01-16 │    18 │ 144-0056  │ 東京都大田区西六郷… │ S13001               │         20161106 │ 0-00000000-0 │ 139.70238 │ 35.54137 │
│ CS001112000029 │ 稲垣 さやか   │ 1         │ 女性   │ 2005-01-24 │    14 │ 144-0056  │ 東京都大田区西六郷… │ S13001               │         20170322 │ 0-00000000-0 │ 139.70238 │ 35.54137 │
│ CS001112000030 │ 内藤 サンタ … │ 1         │ 女性   │ 2003-03-02 │    16 │ 144-0056  │ 東京都大田区西六郷… │ S13001               │         20170611 │ 0-00000000-0 │ 139.70238 │ 35.54137 │
│ CS001113000004 │ 葛西 莉央     │ 1         │ 女性   │ 2003-02-22 │    16 │ 144-0056  │ 東京都大田区西六郷… │ S13001               │         20151105 │ 7-20100308-1 │ 139.70238 │ 35.54137 │
│ CS001113000010 │ 大谷 美菜     │ 1         │ 女性   │ 2005-05-09 │    13 │ 144-0056  │ 東京都大田区西六郷… │ S13001               │         20151120 │ 0-00000000-0 │ 139.70238 │ 35.54137 │
└────────────────┴───────────────┴───────────┴────────┴────────────┴───────┴───────────┴─────────────────────┴──────────────────────┴──────────────────┴──────────────┴───────────┴──────────┘

---
> P-086: 085で作成した緯度経度つき顧客データに対し、会員申込店舗コード（application_store_cd）をキーに店舗データ（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客住所（address）の緯度・経度を用いて申込み店舗と顧客住所の距離（単位：km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は以下の簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示せよ。

$$
\begin{aligned}
& longitude(radian)：\phi \\
& latitude(radian)：\lambda \\
& distance, L = 6371 * \arccos(\sin \phi_1 * \sin \phi_2 + \cos \phi_1 * \cos \phi_2 * \cos(\lambda_1 − \lambda_2))
\end{aligned}
$$

---
> P-087: 顧客データ（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなして1顧客1レコードとなるように名寄せした名寄顧客データを作成し、顧客データの件数、名寄顧客データの件数、重複数を算出せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残し、売上金額合計が同一もしくは売上実績がない顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

In [92]:
df_087 = (
    df_receipt
    .group_by("customer_id")
    .aggregate(sum_amount=_["amount"].sum())
    .join(df_customer, how="outer", predicates="customer_id")
    .drop("customer_id_right")
    .filter(~_["customer_id"].startswith("Z"))
    .order_by([ibis.desc("sum_amount"), "postal_cd"])
    .distinct(on=["customer_name", "postal_cd"], keep="first")
)

df_087.head()

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ customer_id    ┃ sum_amount ┃ customer_name ┃ gender_cd ┃ gender ┃ birth_day  ┃ age   ┃ postal_cd ┃ address                 ┃ application_store_cd ┃ application_date ┃ status_cd    ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ string         │ int64      │ string        │ string    │ string │ date       │ int64 │ string    │ string                  │ string               │ int64            │ string       │
├────────────────┼────────────┼───────────────┼───────────┼────────┼────────────┼───────┼───────────┼─────────────────────────┼──────────────────────┼──────────────────┼──────────────┤
│ CS005513000417 │       2338 │ 葛西 花       │ 1         │ 女性   │ 1963-03-03 │    56 │ 167-0031  │ 東京都杉並区本天沼****… │ S13005               │         20170729 │ A-20090810-2 │
│ CS038415000180 │       2336 │ 砂川 真帆     │ 1         │ 女性   │ 1976-05-05 │    42 │ 134-0084  │ 東京都江戸川区東葛西**… │ S13038               │         20150713 │ A-20101013-D │
│ CS037412000209 │       2335 │ 萩原 季衣     │ 1         │ 女性   │ 1977-09-30 │    41 │ 136-0073  │ 東京都江東区北砂******… │ S13037               │         20151126 │ A-20100523-4 │
│ CS006315000159 │       2335 │ 栗田 朝香     │ 1         │ 女性   │ 1984-05-03 │    34 │ 224-0053  │ 神奈川県横浜市都筑区池… │ S14006               │         20150517 │ A-20080215-2 │
│ CS008513000153 │       2328 │ 杉山 恵梨香   │ 1         │ 女性   │ 1963-02-05 │    56 │ 201-0005  │ 東京都狛江市岩戸南****… │ S13008               │         20150312 │ A-20090615-1 │
└────────────────┴────────────┴───────────────┴───────────┴────────┴────────────┴───────┴───────────┴─────────────────────────┴──────────────────────┴──────────────────┴──────────────┘

---
> P-088: 087で作成したデータを元に、顧客データに統合名寄IDを付与したデータを作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定
> 
> 顧客IDのユニーク件数と、統合名寄IDのユニーク件数の差も確認すること。

In [93]:
df_088 = (
    df_customer
    .join(
        df_087.select("customer_name", "postal_cd", "customer_id"),
        how="inner", predicates=("customer_name", "postal_cd")
    )
    .rename({"integration_id": "customer_id_right"})
)

diff = df_088["customer_id"].nunique() - df_088["integration_id"].nunique()

print("顧客IDのユニーク件数:", df_088["customer_id"].nunique().execute())
print("統合名寄IDのユニーク件数の差:", diff.execute())

顧客IDのユニーク件数: 8320
統合名寄IDのユニーク件数の差: 18


---
> P-089: 売上実績がある顧客を、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

In [94]:
# ibis dataframeだと分けられないのでpandas
df_089 = (
    df_receipt
    .group_by("customer_id")
    .aggregate(sum_amount=_["amount"].sum())
    .filter(_["sum_amount"] > 0)
    .to_pandas()
)

df_train, df_test = train_test_split(df_089, test_size=0.2, random_state=71)
print(f"学習データ割合: {len(df_train)/len(df_089)}")
print(f"テストデータ割合: {len(df_test)/len(df_089)}")
display(df_test.head())

学習データ割合: 0.7999277717587576
テストデータ割合: 0.20007222824124232


,customer_id,sum_amount
5855,CS004505000030,3110
5400,CS017515000174,3769
5938,CS020515000021,3419
1292,CS035215000038,5128
4476,CS013513000016,2592


---
> P-090: レシート明細データ（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月の時系列モデル構築用データを3セット作成せよ。

In [95]:
df_090 = (
    df_receipt
    .mutate(sales_ym=_["sales_ymd"].cast(str)[0:6])
    .group_by("sales_ym")
    .aggregate(sum_amount=_["amount"].sum())
    .order_by("sales_ym")
)

def split_data(
    df,
    train_size: int,
    test_size: int,
    slide_window: int,
    start_point: int,
):
    train_start = start_point * slide_window
    test_start = train_start + train_size
    return df[train_start:test_start], df[test_start : test_start + test_size]


df_090_train1, df_090_test1 = split_data(
    df_090, train_size=12, test_size=6, slide_window=6, start_point=0
)

df_090_train2, df_090_test2 = split_data(
    df_090, train_size=12, test_size=6, slide_window=6, start_point=1
)

df_090_train3, df_090_test3 = split_data(
    df_090, train_size=12, test_size=6, slide_window=6, start_point=2
)

display(df_090_train1.head())

┏━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ sales_ym ┃ sum_amount ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━┩
│ string   │ int64      │
├──────────┼────────────┤
│ 201701   │     902056 │
│ 201702   │     764413 │
│ 201703   │     962945 │
│ 201704   │     847566 │
│ 201705   │     884010 │
└──────────┴────────────┘

In [96]:
display(df_090_test1.head())

┏━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ sales_ym ┃ sum_amount ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━┩
│ string   │ int64      │
├──────────┼────────────┤
│ 201801   │     944509 │
│ 201802   │     864128 │
│ 201803   │     946588 │
│ 201804   │     937099 │
│ 201805   │    1004438 │
└──────────┴────────────┘

In [97]:
df_090_train2.head()

┏━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ sales_ym ┃ sum_amount ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━┩
│ string   │ int64      │
├──────────┼────────────┤
│ 201707   │     959205 │
│ 201708   │     954836 │
│ 201709   │     902037 │
│ 201710   │     905739 │
│ 201711   │     932157 │
└──────────┴────────────┘

---
> P-091: 顧客データ（df_customer）の各顧客に対し、売上実績がある顧客数と売上実績がない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

In [98]:
from imblearn.under_sampling import RandomUnderSampler

df_091 = (
    df_receipt
    .group_by("customer_id")
    .aggregate(sum_amount=_["amount"].sum())
    .join(df_customer, how="right", predicates="customer_id")
    .mutate(is_buy_flag=_["sum_amount"].isnull())
    .to_pandas()
)

rs = RandomUnderSampler(random_state=71)
df_091, _ = rs.fit_resample(df_091, df_091["is_buy_flag"])
print('0の件数', len(df_091[df_091["is_buy_flag"] == 0]))
print('1の件数', len(df_091[df_091["is_buy_flag"] == 1]))

0の件数 8306
1の件数 8306


---
> P-092: 顧客データ（df_customer）の性別について、第三正規形へと正規化せよ。

- 第1正規形：1つのセルには1つの値しか含まれない　list型などで登録しない
- 第2正規形：部分関数従属を排除し、完全関数従属にする　Nullとか無くす
- 第3正規形：第2正規形のテーブルから、推移的関数従属している列が切り出されたもの　段階的な従属を無くす

In [99]:
df_092_gender_std = (
    df_customer
    .select("gender_cd", "gender")
    .distinct()
)


df_092_customer_std = df_customer.drop("gender")

df_092_gender_std

┏━━━━━━━━━━━┳━━━━━━━━┓
┃ gender_cd ┃ gender ┃
┡━━━━━━━━━━━╇━━━━━━━━┩
│ string    │ string │
├───────────┼────────┤
│ 9         │ 不明   │
│ 1         │ 女性   │
│ 0         │ 男性   │
└───────────┴────────┘

In [100]:
df_092_customer_std.head()

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ customer_id    ┃ customer_name ┃ gender_cd ┃ birth_day  ┃ age   ┃ postal_cd ┃ address                ┃ application_store_cd ┃ application_date ┃ status_cd    ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ string         │ string        │ string    │ date       │ int64 │ string    │ string                 │ string               │ int64            │ string       │
├────────────────┼───────────────┼───────────┼────────────┼───────┼───────────┼────────────────────────┼──────────────────────┼──────────────────┼──────────────┤
│ CS021313000114 │ 大野 あや子   │ 1         │ 1981-04-29 │    37 │ 259-1113  │ 神奈川県伊勢原市粟窪*… │ S14021               │         20150905 │ 0-00000000-0 │
│ CS037613000071 │ 六角 雅彦     │ 9         │ 1952-04-01 │    66 │ 136-0076  │ 東京都江東区南砂*****… │ S13037               │         20150414 │ 0-00000000-0 │
│ CS031415000172 │ 宇多田 貴美子 │ 1         │ 1976-10-04 │    42 │ 151-0053  │ 東京都渋谷区代々木***… │ S13031               │         20150529 │ D-20100325-C │
│ CS028811000001 │ 堀井 かおり   │ 1         │ 1933-03-27 │    86 │ 245-0016  │ 神奈川県横浜市泉区和 … │ S14028               │         20160115 │ 0-00000000-0 │
│ CS001215000145 │ 田崎 美紀     │ 1         │ 1995-03-29 │    24 │ 144-0055  │ 東京都大田区仲六郷***… │ S13001               │         20170605 │ 6-20090929-2 │
└────────────────┴───────────────┴───────────┴────────────┴───────┴───────────┴────────────────────────┴──────────────────────┴──────────────────┴──────────────┘

---
> P-093: 商品データ（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータ（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データを作成せよ。

In [101]:
df_093 = (
    df_product
    .join(
        df_category.select(
            "category_small_cd",
            "category_major_name",
            "category_medium_name",
            "category_small_name",
        ),
        how="left", predicates="category_small_cd"
    )
)

df_093.head()

┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ product_cd ┃ category_major_cd ┃ category_medium_cd ┃ category_small_cd ┃ unit_price ┃ unit_cost ┃ category_small_cd_right ┃ category_major_name ┃ category_medium_name ┃ category_small_name ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ string     │ string            │ string             │ string            │ int64      │ int64     │ string                  │ string              │ string               │ string              │
├────────────┼───────────────────┼────────────────────┼───────────────────┼────────────┼───────────┼─────────────────────────┼─────────────────────┼──────────────────────┼─────────────────────┤
│ P040101001 │ 04                │ 0401               │ 040101            │        198 │       149 │ 040101                  │ 惣菜                │ 御飯類               │ 弁当類              │
│ P040101002 │ 04                │ 0401               │ 040101            │        218 │       164 │ 040101                  │ 惣菜                │ 御飯類               │ 弁当類              │
│ P040101003 │ 04                │ 0401               │ 040101            │        230 │       173 │ 040101                  │ 惣菜                │ 御飯類               │ 弁当類              │
│ P040101004 │ 04                │ 0401               │ 040101            │        248 │       186 │ 040101                  │ 惣菜                │ 御飯類               │ 弁当類              │
│ P040101005 │ 04                │ 0401               │ 040101            │        268 │       201 │ 040101                  │ 惣菜                │ 御飯類               │ 弁当類              │
└────────────┴───────────────────┴────────────────────┴───────────────────┴────────────┴───────────┴─────────────────────────┴─────────────────────┴──────────────────────┴─────────────────────┘

## ファイル入出力

---
> P-094: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること
> 
> |出力先|
> |:--:|
> |./data|

In [102]:
df_093.to_csv(
    "../data/P_df_093_UTF-9_header.csv",
)

---
> P-095: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|CP932|
> 
> ファイル出力先のパスは以下のようにすること。
> 
> |出力先|
> |:--:|
> |./data|

---
> P-096: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|無し|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること。
> 
> |出力先|
> |:--:|
> |./data|

---
> P-097: 094で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|

---
> P-098: 096で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|ヘッダ無し|UTF-8|

---
> P-099: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること
> 
> |出力先|
> |:--:|
> |./data|

In [103]:
# separatorでタブで分割


---
> P-100: 099で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|

問題はここで終了です。お疲れ様でした。